# GF - MIDID - GATEWAY Project Conclusion

REPEAT: N in both days 



NEW: Y yesterday and N Today 

Here's the step-by-step plan for processing the credit data:

1. **Extract the central dataframe**
   - If teh files have as columns 'Details', 'Posting Date', 'Description', 'Amount', 'Type', 'Balance',
       'Check or Slip #', we will add up all the rows onto each other to make one big file.
   - Once the joined up file is created, we will filter on Credits.
   - Implement debugging:
     1. Check if the dataframe has the exact same number of rows and columns as the totals for the per corp one.✅
     2. Check if unique values are Credits & Debits in  the the column "Type".✅

2. **Extract GatewayMIDID-**
   - Start by creating the dictionary where we have as keys Gateway IDs and as keys Mid IDs.✅
   - We can do this by importing the dataframe sent in the Automnation chat.✅
   - Implement debugging:
     1. Check if the dataframe where we got the dict from has the same amount of rows as the excel sheet.✅
     2. Check if at keast one of the midids are present in each row + create an indexation of the rows with a matched ID.✅
     3. If the length of the index doesn´t match the number of rows in the dataframe, we visualize the rows that were missed.✅

3. **Get all the unique credit charges in the dataset**
   - Get all the unique credit charges in the credit dataset✅
   - Check if these rows contain at least one of these + add the matching rows index to a dict.✅

4. **Create the backbone dict**
   - Keys: unique credit types ✅
   - Values: Dict ✅
   - Dict has as Keys: Unique MidIDS of each transaction✅
   - Dict has Values: for now 0 (will be populated) ✅

5. **Populate the Backbone Dataframe + Convert to Gateway IDs**
   - Iterate the credit transactions
   - Create an index tracker of all the transactions that have a credit type and MIDID match.
   - Once this is done, we map the column names with the GateIDS
   - Finito

In [1]:
import glob
import pandas as pd
import numpy as np 
import os
import sys
import glob
import datetime

def clean_column_names(df):
    df.columns = (
        df.columns.str.strip()         # Remove leading/trailing spaces
        .str.lower()                   # Convert to lowercase
        .str.replace(r'\W+', '_', regex=True)  # Replace non-word characters with '_'
        .str.replace(r'_+', '_', regex=True)   # Remove multiple consecutive '_'
        .str.rstrip('_')                # Remove trailing '_'
    )
    return df

# Get a function that will clean all the strings 
def clean_string(text):
    """
    Clean a string by removing special characters and spaces, converting to lowercase
    
    Args:
        text (str): The input string to clean
        
    Returns:
        str: Cleaned string with only alphanumeric characters in lowercase
    """
    import re
    # Remove special characters and spaces, convert to lowercase
    cleaned = re.sub(r'[^a-zA-Z0-9]', '', text).lower()
    return cleaned

### Get the correct corp names

In [2]:
# Import Deposit-Reconciliation_EXPORT xlsx
folder_path = os.getcwd()  # gets the folder where the notebook is located

# Get a function that will clean all the strings 
def clean_string(text):
    """
    Clean a string by removing special characters, spaces, and numbers, converting to lowercase
    
    Args:
        text (str): The input string to clean
        
    Returns:
        str: Cleaned string with only alphabetic characters in lowercase
    """
    import re
    # Remove special characters, numbers, and spaces, convert to lowercase
    cleaned = re.sub(r'[^a-zA-Z]', '', text).lower()
    return cleaned

def find_deposit_file(folder_path, base_words=["deprec", "export"]):
    # Match any csv/xls/xlsx containing the keywords
    all_files = os.listdir(folder_path)
    matching_files = []
    
    for file in all_files:
        # Only check files with csv or excel extensions
        if file.lower().endswith(('.csv', '.xls', '.xlsx')):
            file_clean = clean_string(file)
            # Check if all required words are in the cleaned filename
            if all(word in file_clean for word in base_words):
                matching_files.append(os.path.join(folder_path, file))
                print(f"Found matching file: {file}")
    
    if not matching_files:
        raise FileNotFoundError(f"No matching files found in {folder_path} containing {base_words}")
        
    # Sort by most recent modified time
    matching_files.sort(key=os.path.getmtime, reverse=True)
    return matching_files[0]  # return the newest one

# Now load it
file_path = find_deposit_file(folder_path)
print(f"📄 Loading file: {file_path}")

# Load the file with previous flexible loader
def load_file(file_path):
    """
    Load a file (CSV or Excel) with appropriate engine settings.
    
    Args:
        file_path (str): Path to the file to load
        
    Returns:
        pandas.DataFrame: Loaded data
    """
    ext = os.path.splitext(file_path)[1].lower()
    if ext == '.csv':
        return pd.read_csv(file_path)
    elif ext in ['.xls', '.xlsx']:
        return pd.read_excel(file_path, engine='openpyxl')
    else:
        raise ValueError(f"Unsupported file type: {ext}") 

df = load_file(file_path)

unique_corps = df['Merchant Group'].unique()

# Remove any non strings 
unique_corps = [corp for corp in unique_corps if isinstance(corp, str)]

# Make the unique corp list in lowercase
unique_corps = [corp.lower() for corp in unique_corps]
short_corps = []
for corp in unique_corps:
    # Split corp on space 
    corp_clean = corp.lower().replace(" ", "")
    short_corps.append(corp_clean)
short_corps



Found matching file: DEPREC_EXPORT_1750684702.csv
📄 Loading file: c:\Users\mmsou\Documents\mokum.ai\Goldfinger\DEPREC_EXPORT_1750684702.csv


['brightadvantagellc',
 'diamondspherellc',
 'primesmartsolutionsllc',
 'salesshield',
 'specificonllc',
 'uniqueplusllc']

In [3]:
current_directory = os.getcwd()
all_files = os.listdir(current_directory)
# Get list of valid CSV files
import os
from rapidfuzz import fuzz

exact_csv_dict = {}

for corp in short_corps:
    list_of_scores = []
    
    for file in all_files:
        score = fuzz.ratio(corp, file)
        list_of_scores.append((file,score))
    
    selected_file = ""
    selected_file = max(list_of_scores, key=lambda x: x[1])
    if selected_file[1] > 35:
        exact_csv_dict[corp] = selected_file[0]

exact_csv_dict

{'brightadvantagellc': 'bright.CSV',
 'diamondspherellc': 'diamond.CSV',
 'primesmartsolutionsllc': 'prime.csv',
 'specificonllc': 'specificon.CSV',
 'uniqueplusllc': 'unique.CSV'}

## 1 - Central dataframe

### 1.1 - Get the different types of csvs / diffrent types of bank statements

In [4]:
current_directory = os.getcwd()
all_files = os.listdir(current_directory)
# Get list of valid CSV files
BankStatementList = []

for key, val in exact_csv_dict.items():
    BankStatementList.append(val)
print(BankStatementList)

import pandas as pd

def find_header_row(filename, header_keywords=["Date", "Description", "Amount"]):
    with open(filename, 'r', encoding='utf-8') as f:
        for i, line in enumerate(f):
            # Check if all keywords are in the line (case-insensitive)
            if all(keyword.lower() in line.lower() for keyword in header_keywords):
                return i
    return None
corp_dfs = {}  # <-- Add this before your for loop

for file in BankStatementList:
    if file.lower().endswith('.csv'):
        header_row = find_header_row(file)
        if header_row is not None:
            df = pd.read_csv(file, skiprows=header_row)
            base_name = os.path.splitext(os.path.basename(file))[0].replace(" ", "_")
            corp_dfs[base_name] = df
            print(f"Loaded {base_name} from {file}")
        else:
            print(f"Could not find header in {file}")



['bright.CSV', 'diamond.CSV', 'prime.csv', 'specificon.CSV', 'unique.CSV']
Loaded bright from bright.CSV
Loaded diamond from diamond.CSV
Loaded prime from prime.csv
Loaded specificon from specificon.CSV
Loaded unique from unique.CSV


In [5]:
import pandas as pd
import os

# Your find_header_row function is correct and necessary.
def find_header_row(filename, header_keywords=["Date", "Description", "Amount"]):
    """Finds the first row containing all header keywords."""
    try:
        with open(filename, 'r', encoding='utf-8', errors='ignore') as f:
            for i, line in enumerate(f):
                if all(keyword.lower() in line.lower() for keyword in header_keywords):
                    return i
    except Exception as e:
        print(f"    Error reading {filename} to find header: {e}")
    return None

# The final, correct column names to be used if a header isn't found
final_column_names = ["Details", "Posting Date", "Description", "Amount", "Type", "Balance", "Check or Slip #"]

corp_dfs = {}
print("--- Starting Robust File Import ---")

# Use your BankStatementList from the previous cell
for file in BankStatementList:
    if file.lower().endswith('.csv'):
        print(f"\nProcessing file: {file}")
        header_row_index = find_header_row(file)

        if header_row_index is not None:
            # Case 1: A header was found. Read from that line.
            print(f"    Header found on line {header_row_index + 1}. Reading with explicit index.")
            # The key is adding index_col=False to stop pandas from guessing.
            df = pd.read_csv(
                file, 
                skiprows=header_row_index, 
                index_col=False  # This FORCES pandas to create a new 0,1,2.. index
            )
        else:
            # Case 2: No standard header found. Treat the whole file as data.
            print(f"    No header found. Reading raw data with explicit index.")
            # Here we tell pandas there's no header AND no index column.
            df = pd.read_csv(
                file, 
                header=None, 
                index_col=False # This is the crucial fix
            )
            # Since there was no header, we must assign the column names ourselves.
            num_columns_found = len(df.columns)
            df.columns = final_column_names[:num_columns_found]

        # Get a clean name for the dictionary key
        base_name = os.path.splitext(os.path.basename(file))[0].replace(" ", "_")
        corp_dfs[base_name] = df
        print(f"    Successfully loaded '{base_name}' into DataFrame.")

print("\n--- Robust File Import Complete ---")
# Now, every dataframe in corp_dfs will have a correct, standard index.

--- Starting Robust File Import ---

Processing file: bright.CSV
    Header found on line 1. Reading with explicit index.
    Successfully loaded 'bright' into DataFrame.

Processing file: diamond.CSV
    Header found on line 1. Reading with explicit index.
    Successfully loaded 'diamond' into DataFrame.

Processing file: prime.csv
    Header found on line 7. Reading with explicit index.
    Successfully loaded 'prime' into DataFrame.

Processing file: specificon.CSV
    Header found on line 1. Reading with explicit index.
    Successfully loaded 'specificon' into DataFrame.

Processing file: unique.CSV
    Header found on line 1. Reading with explicit index.
    Successfully loaded 'unique' into DataFrame.

--- Robust File Import Complete ---


In [6]:
length_dict = {}

for corp, df in corp_dfs.items():
    col_len = len(df.columns)
    if col_len not in length_dict:
        length_dict[col_len] = []
    length_dict[col_len].append(corp)

print(length_dict)

{7: ['bright', 'diamond', 'specificon', 'unique'], 4: ['prime']}


## **Type 4 - Apex & Dori**

In [7]:
for ncolumns in length_dict.keys():
    if ncolumns == 4:
        print(length_dict[ncolumns])

columns_needed = [
    "Details",
    "Posting Date",
    "Description",
    "Amount",
    "Type",
    "Balance",
    "Check or Slip #"
]

['prime']


In [8]:
# Create a fucntion that cleans all coprs that have statements with 4 columns
import numpy as np

def adapt_4_columns(list_of_csv_with_4_cols, corp_dfs):
    x = list_of_csv_with_4_cols
    print(x)

    result_dfs = {}

    for corp in x:
        for name, df in corp_dfs.items():
            if corp in name:
                df = corp_dfs[name]
                print(f"🚨 Corp in question: {corp}")

                # Now we have each of the dfs that have 4 columns
                df["Amount"] = df["Amount"].astype(float)
                is_float = np.issubdtype(df["Amount"].dtype, np.floating)
                print(f"Is the column Amount in the float type for both dfs? {is_float}")

                # Create a column called Details based on the Amount column, slapping Credit or Debit in its cells
                df["Details"] = df["Amount"].apply(lambda x: "DEBIT" if x < 0 else "CREDIT")
                print(f"Do the columns in each df as unique values CREDIT and DEBIT?: {'DEBIT' in df['Details'].unique() or 'CREDIT' in df['Details'].unique()}")

                # Create a new column called Type using the same logic but here we use ACH_CREDIT or debit
                df["Type"] = df["Amount"].apply(lambda x: "ACH_DEBIT" if x < 0 else "ACH_CREDIT")
                print(f"Do the columns in each df as unique values ACH_CREDIT and ACH_DEBIT?: {'ACH_DEBIT' in df['Type'].unique() or 'ACH_CREDIT' in df['Type'].unique()}")

                # Create the two empty Balance and Check or Slip # 
                df["Check or Slip #"] = ""
                df["Running Bal."] = ""


                # Change the Date column to Posting Date
                df = df.rename(columns={"Date": "Posting Date"})
                df = df.rename(columns={"Running Bal.": "Balance"})
                print(f"The columns in each of the dfs are {df.columns}")

                # Check if all columns_needed are present
                missing = [col for col in columns_needed if col not in df.columns]
                if missing:
                    print(f"{name} is missing columns: {missing}")
                else:
                    print(f"{name} has all required columns.")

                df = df[columns_needed]

                # Now comes in the main part which is to change the descripsection to match the NEF filesdf = df[columns_needed]
                df['Description'] = df['Description'].str.replace('REL ID:', 'IND ID:', regex=False).str.replace('DEP ID:', 'IND ID:', regex=False) 
                df['Description'] = df['Description'].str.replace('INDN', 'IND NAME', regex=False)
                df['Description'] = df['Description'].str.replace('CHB ID', 'IND NAME', regex=False)
                df['Description'] = df['Description'].str.replace(' CHBK ID:', 'IND ID', regex=False)
                if 'Balance' in df.columns:
                        df = df.loc[:, ~df.columns.duplicated()] 

                df = df.dropna(how='all')

                result_dfs[name] = df

    return result_dfs

# Usage:
# processed_dfs = adapt_4_columns(length_dict[4], corp_dfs)
dfs_4 = adapt_4_columns(length_dict[4], corp_dfs)
dfs_4

['prime']
🚨 Corp in question: prime
Is the column Amount in the float type for both dfs? True
Do the columns in each df as unique values CREDIT and DEBIT?: True
Do the columns in each df as unique values ACH_CREDIT and ACH_DEBIT?: True
The columns in each of the dfs are Index(['Posting Date', 'Description', 'Amount', 'Balance', 'Details', 'Type',
       'Check or Slip #'],
      dtype='object')
prime has all required columns.


{'prime':    Details Posting Date                                        Description  \
 0   CREDIT    6/20/2025                 Beginning balance as of 06/20/2025   
 1   CREDIT    6/20/2025  BANKCARD DES:MTOT IND ID:422899490017267 IND N...   
 2   CREDIT    6/20/2025  BANKCARD DEP DES:MERCH IND ID:739788746585208 ...   
 3   CREDIT    6/20/2025  MERCHANT SERVICE DES:MERCH IND ID:180668612303...   
 4   CREDIT    6/20/2025  MERCHANT SERVICE DES:MERCH IND ID:180619392303...   
 5   CREDIT    6/20/2025  PAYARC DES:TP RES IND ID:567000000870345 IND N...   
 6   CREDIT    6/20/2025  MERCHANT SERVICE DES:MERCH IND ID:180668612303...   
 7   CREDIT    6/20/2025  PAYARC DES:TP RES IND ID:567000000870345 IND N...   
 8   CREDIT    6/20/2025  BANKCARD DEP DES:MERCH IND ID:739729764950577 ...   
 9   CREDIT    6/20/2025  MERCHANT SERVICE DES:MERCH IND ID:180142512303...   
 10  CREDIT    6/20/2025  BANKCARD DEP DES:MERCH IND ID:739729764950577 ...   
 11  CREDIT    6/20/2025  MERCHANT SERVICE 

### Now that we have the different corp sizes, we know how to act accordingly, such that in the end we can combine all these to one major df

## **Type 7 - Piterias, Jupiter & Silvers** 

### No we will clean up the most important part which is the description section,we need to set this as siilar as possible to the strucure of the statements that we are presented with in NEF

#### All good here as they are the same format as nef

In [9]:
for ncolumns in length_dict.keys():
    if ncolumns == 7:
        print(length_dict[ncolumns])

columns_needed = [
    "Details",
    "Posting Date",
    "Description",
    "Amount",
    "Type",
    "Balance",
    "Check or Slip #"
]

['bright', 'diamond', 'specificon', 'unique']


In [10]:
import numpy as np

def adapt_4_columns(list_of_csv_with_4_cols, corp_dfs):
    x = list_of_csv_with_4_cols

    result_dfs = {}

    for corp in x:
        for name, df in corp_dfs.items():
            if corp in name:
                df = corp_dfs[name]
                print(f"🚨 Corp in question: {corp}")
                print(df.columns)
                df = df.dropna(how='all')

                result_dfs[name] = df

    return result_dfs




dfs_7 = adapt_4_columns(length_dict[7], corp_dfs)


🚨 Corp in question: bright
Index(['Details', 'Posting Date', 'Description', 'Amount', 'Type', 'Balance',
       'Check or Slip #'],
      dtype='object')
🚨 Corp in question: diamond
Index(['Details', 'Posting Date', 'Description', 'Amount', 'Type', 'Balance',
       'Check or Slip #'],
      dtype='object')
🚨 Corp in question: specificon
Index(['Details', 'Posting Date', 'Description', 'Amount', 'Type', 'Balance',
       'Check or Slip #'],
      dtype='object')
🚨 Corp in question: unique
Index(['Details', 'Posting Date', 'Description', 'Amount', 'Type', 'Balance',
       'Check or Slip #'],
      dtype='object')


## **Type 19 - PromotionKnights, Advertising_Space, Today_and_Tomorrow_Promos_LLC** 

#### These will be ignored as they do not include mid ids

### Now we will make a dict that has all the corps that we want

In [11]:
needed_dict = {}

# First add everything from dfs_4
for k, v in dfs_4.items():
    needed_dict[k] = v

# Then add everything from dfs_7
for k, v in dfs_7.items():
    needed_dict[k] = v  # if key exists, this will overwrite with dfs_7 value

needed_dict.keys()

for key, val in needed_dict.items():
    print(key, val.columns)

needed_dict

prime Index(['Details', 'Posting Date', 'Description', 'Amount', 'Type', 'Balance',
       'Check or Slip #'],
      dtype='object')
bright Index(['Details', 'Posting Date', 'Description', 'Amount', 'Type', 'Balance',
       'Check or Slip #'],
      dtype='object')
diamond Index(['Details', 'Posting Date', 'Description', 'Amount', 'Type', 'Balance',
       'Check or Slip #'],
      dtype='object')
specificon Index(['Details', 'Posting Date', 'Description', 'Amount', 'Type', 'Balance',
       'Check or Slip #'],
      dtype='object')
unique Index(['Details', 'Posting Date', 'Description', 'Amount', 'Type', 'Balance',
       'Check or Slip #'],
      dtype='object')


{'prime':    Details Posting Date                                        Description  \
 0   CREDIT    6/20/2025                 Beginning balance as of 06/20/2025   
 1   CREDIT    6/20/2025  BANKCARD DES:MTOT IND ID:422899490017267 IND N...   
 2   CREDIT    6/20/2025  BANKCARD DEP DES:MERCH IND ID:739788746585208 ...   
 3   CREDIT    6/20/2025  MERCHANT SERVICE DES:MERCH IND ID:180668612303...   
 4   CREDIT    6/20/2025  MERCHANT SERVICE DES:MERCH IND ID:180619392303...   
 5   CREDIT    6/20/2025  PAYARC DES:TP RES IND ID:567000000870345 IND N...   
 6   CREDIT    6/20/2025  MERCHANT SERVICE DES:MERCH IND ID:180668612303...   
 7   CREDIT    6/20/2025  PAYARC DES:TP RES IND ID:567000000870345 IND N...   
 8   CREDIT    6/20/2025  BANKCARD DEP DES:MERCH IND ID:739729764950577 ...   
 9   CREDIT    6/20/2025  MERCHANT SERVICE DES:MERCH IND ID:180142512303...   
 10  CREDIT    6/20/2025  BANKCARD DEP DES:MERCH IND ID:739729764950577 ...   
 11  CREDIT    6/20/2025  MERCHANT SERVICE 

In [12]:
needed_dict.keys()

dict_keys(['prime', 'bright', 'diamond', 'specificon', 'unique'])

### Now that we have the different corp sizes, we know how to act accordingly, such that in the end we can combine all these to one major df

In [13]:
import pandas as pd
import os

# Function to check if all values in a dictionary are the same
all_same = lambda d: len(set(map(tuple, d.values()))) == 1

# Function to clean column names
def clean_column_names(df):
    df.columns = (
        df.columns.str.strip()         # Remove leading/trailing spaces
        .str.lower()                   # Convert to lowercase
        .str.replace(r'\W+', '_', regex=True)  # Replace non-word characters with '_'
        .str.replace(r'_+', '_', regex=True)   # Remove multiple consecutive '_'
        .str.rstrip('_')                # Remove trailing '_'
    )
    return df

# Function to create the central dataframe
def central_dataframe_creator(df_dict):
    # Dict with column names
    column_names_per_corp = {corp: df.columns.tolist() for corp, df in df_dict.items()}

    # Check if all DataFrames have the same columns
    c = all_same(column_names_per_corp)
    print(f"✅ Are all the columns in the dataframes the same? {c}")

    if not c:
        print("⚠️ The columns in the dataframes are not the same, please check the data.")
        raise Exception("⚠️ The columns in the dataframes are not the same, please check the data.")
        return None

    # Create an empty central dataframe
    central_df = pd.DataFrame(columns=column_names_per_corp[list(df_dict.keys())[0]])
    central_df.index.name = "Index"

    # Use concat to combine all DataFrames
    if df_dict:
        central_df = pd.concat(df_dict.values(), ignore_index=True)
    central_df = clean_column_names(central_df)
    central_df = central_df.dropna(how='all')
    

    return central_df  # Return the final DataFrame

# Run the function with your dictionary of DataFrames
test = central_dataframe_creator(needed_dict) 
test.to_csv('central_dataframe.csv', index=True)
test.head(50)

✅ Are all the columns in the dataframes the same? True


,details,posting_date,description,amount,type,balance,check_or_slip
0,CREDIT,6/20/2025,Beginning balance as of 06/20/2025,NaN,ACH_CREDIT,,
1,CREDIT,6/20/2025,BANKCARD DES:MTOT IND ID:422899490017267 IND N...,504.48,ACH_CREDIT,,
2,CREDIT,6/20/2025,BANKCARD DEP DES:MERCH IND ID:739788746585208 ...,388.47,ACH_CREDIT,,
3,CREDIT,6/20/2025,MERCHANT SERVICE DES:MERCH IND ID:180668612303...,384.25,ACH_CREDIT,,
4,CREDIT,6/20/2025,MERCHANT SERVICE DES:MERCH IND ID:180619392303...,364.63,ACH_CREDIT,,
5,CREDIT,6/20/2025,PAYARC DES:TP RES IND ID:567000000870345 IND N...,359.92,ACH_CREDIT,,
6,CREDIT,6/20/2025,MERCHANT SERVICE DES:MERCH IND ID:180668612303...,289.65,ACH_CREDIT,,
7,CREDIT,6/20/2025,PAYARC DES:TP RES IND ID:567000000870345 IND N...,276.61,ACH_CREDIT,,
8,CREDIT,6/20/2025,BANKCARD DEP DES:MERCH IND ID:739729764950577 ...,259.96,ACH_CREDIT,,
9,CREDIT,6/20/2025,MERCHANT SERVICE DES:MERCH IND ID:180142512303...,242.62,ACH_CREDIT,,


In [14]:
test.tail(50)

,details,posting_date,description,amount,type,balance,check_or_slip
176,CREDIT,6/23/2025,ORIG CO NAME:BANKCARD DEP CO ENTRY DESCR:M...,84.20,ACH_CREDIT,,NaN
177,CREDIT,6/23/2025,ORIG CO NAME:MERCHANT SERVICE CO ENTRY DESCR:M...,88.79,ACH_CREDIT,,NaN
178,CREDIT,6/23/2025,ORIG CO NAME:MERCHANT SERVICE CO ENTRY DESCR:M...,93.35,ACH_CREDIT,,NaN
179,CREDIT,6/23/2025,ORIG CO NAME:BANKCARD CO ENTRY DESCR:M...,116.49,ACH_CREDIT,,NaN
180,CREDIT,6/23/2025,ORIG CO NAME:BANKCARD DEP CO ENTRY DESCR:M...,161.15,ACH_CREDIT,,NaN
181,CREDIT,6/23/2025,ORIG CO NAME:BANKCRD CO ENTRY DESCR:C...,242.63,ACH_CREDIT,,NaN
182,CREDIT,6/23/2025,ORIG CO NAME:BANKCRD CO ENTRY DESCR:C...,275.61,ACH_CREDIT,,NaN
183,CREDIT,6/23/2025,ORIG CO NAME:BANKCARD DEP CO ENTRY DESCR:M...,406.86,ACH_CREDIT,,NaN
184,CREDIT,6/23/2025,ORIG CO NAME:BANKCRD CO ENTRY DESCR:C...,436.17,ACH_CREDIT,,NaN
185,CREDIT,6/23/2025,ORIG CO NAME:MERCHANT SERVICE CO ENTRY DESCR:M...,464.20,ACH_CREDIT,,NaN


### Now that we have the different corp sizes, we know how to act accordingly, such that in the end we can combine all these to one major df

In [15]:
# Check if all rows are Credits or Debits
unique_details = test["details"].unique()
print("🔍 Unique values in 'Details' column:", unique_details)

# Check this outlier
df_dslip = test[test["details"] == "DSLIP"]
print("Here we have teh column/s where we dont have a Credit or Debit:")
print(df_dslip)

# Create the Credit datase
credit_data = test[test["details"].isin(["CREDIT"])]
credit_data["clean_description"] = (
    credit_data["description"]
    .str.lower()  # Convert to lowercase
    .str.replace(r'\s+', '', regex=True)  # Remove ALL spaces (including between words)
    .str.replace(r'[^a-zA-Z0-9]', '', regex=True)  # Remove special characters
)
credit_data.shape



🔍 Unique values in 'Details' column: ['CREDIT' 'DEBIT']
Here we have teh column/s where we dont have a Credit or Debit:
Empty DataFrame
Columns: [details, posting_date, description, amount, type, balance, check_or_slip]
Index: []


C:\Users\mmsou\AppData\Local\Temp\ipykernel_36028\1249047335.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  credit_data["clean_description"] = (


(166, 8)

## 2 - Get the dictionary where we have as keys GatewayIDs and values MIDIDs, & the one where we have Processor as keys and Gateway as Value

In [16]:
def clean_column_names(df):
    df.columns = (
        df.columns.str.strip()         # Remove leading/trailing spaces
        .str.lower()                   # Convert to lowercase
        .str.replace(r'\W+', '_', regex=True)  # Replace non-word characters with '_'
        .str.replace(r'_+', '_', regex=True)   # Remove multiple consecutive '_'
        .str.rstrip('_')                # Remove trailing '_'
    )
    return df
import pandas as pd

# Load the Excel file (Ensure the correct file extension)
gwids_df = pd.read_excel("GWID_local_GF.xlsx", engine='openpyxl')  # Change to .xls if needed

# Apply a function to clean column names (if needed)
gwids_df = clean_column_names(gwids_df)

# Strip whitespace and ensure all values are strings using `.map()`
gwids_df = gwids_df.map(lambda x: str(x).strip() if pd.notna(x) else x)

# Create the GWID dictionary with cleaned values and remove leading zeros from MID ID
GWID_dict = {
    str(row.gwid).strip(): str(int(row.mid.strip())) if row.mid.strip().isdigit() else str(row.mid).strip()
    for row in gwids_df.itertuples(index=False)
}

# Print a sample of GWID_dict (Gateway ID → MID ID)
print("🔹 GWID_dict (Gateway ID → MID ID) Mapping:")

# Show only first 10 mappings for readability
for gwid, mid in list(GWID_dict.items()):  
    print(f"🔹 Gateway ID: {gwid} → MID ID: {mid}")

# Check the total number of mappings
print(f"Total mappings in GWID_dict: {len(GWID_dict)}")


🔹 GWID_dict (Gateway ID → MID ID) Mapping:
🔹 Gateway ID: 2 → MID ID: 567000000859181
🔹 Gateway ID: 3 → MID ID: 9180621072288880
🔹 Gateway ID: 4 → MID ID: 9180363862288840
🔹 Gateway ID: 5 → MID ID: 9180454822288910
🔹 Gateway ID: 6 → MID ID: 9180941172288810
🔹 Gateway ID: 7 → MID ID: 567000000864041
🔹 Gateway ID: 8 → MID ID: 567000000864025
🔹 Gateway ID: 9 → MID ID: 8739744451333230
🔹 Gateway ID: 10 → MID ID: 8739724272527570
🔹 Gateway ID: 11 → MID ID: 8739780712387270
🔹 Gateway ID: 12 → MID ID: 8739732218289060
🔹 Gateway ID: 13 → MID ID: 8739768915392320
🔹 Gateway ID: 14 → MID ID: 8739740872762050
🔹 Gateway ID: 15 → MID ID: 7620306740163700
🔹 Gateway ID: 16 → MID ID: 7620869500163710
🔹 Gateway ID: 25 → MID ID: 934600000133389
🔹 Gateway ID: 26 → MID ID: 934600000133371
🔹 Gateway ID: 27 → MID ID: 934600000133363
🔹 Gateway ID: 28 → MID ID: 934600000133355
🔹 Gateway ID: 21 → MID ID: 934600000133462
🔹 Gateway ID: 22 → MID ID: 934600000133454
🔹 Gateway ID: 23 → MID ID: 934600000133470
🔹 Gatew

### 2.2. Dict with Processor as keys and Gateway as Values

In [17]:
unique_processor_list = gwids_df["processor"].unique()
print("Unique Processor List:", unique_processor_list)

processor_gw_dict = {processor: [] for processor in unique_processor_list}

# Iterate through the dataframe and fill in the values
for row in gwids_df.itertuples(index=False):
    processor_gw_dict[row.processor].append(str(row.gwid))  # Store as list of strings

# Convert lists to sets (to remove duplicates, if any)
processor_gw_dict = {k: set(v) for k, v in processor_gw_dict.items()}

# Print sample output to check
for processor, gwids in list(processor_gw_dict.items())[:5]:  # Show only 5 samples
    print(f"🔹 {processor}: {gwids}")

Unique Processor List: ['Evolve (PayArc)' 'Esquire (Netevia)' 'Synovus'
 'Signapay (Esquire)(Travis)' 'QUANTUM (CBSL) (JORDAN)'
 'APPS(SYNOVUS)(PLUSH PAY)' 'PAYSAFE(PNC)(PLUSH PAY)'
 'PAYSAFE(BBVA)(PLUSH PAY)' 'PAYSAFE (BBVA)(PLUSH PAY)']
🔹 Evolve (PayArc): {'7', '2', '49', '48', '8', '40', '89', '38', '73'}
🔹 Esquire (Netevia): {'6', '63', '42', '62', '55', '3', '37', '34', '57', '64', '39', '58', '41', '35', '61', '56', '4', '43', '44', '5'}
🔹 Synovus: {'51', '11', '59', '52', '13', '53', '50', '9', '30', '10', '14', '29', '60', '36', '12'}
🔹 Signapay (Esquire)(Travis): {'69', '16', '18', '19', '15', '68', '17'}
🔹 QUANTUM (CBSL) (JORDAN): {'77', '27', '28', '75', '47', '70', '94', '22', '93', '23', '91', '81', '92', '45', '25', '76', '24', '46', '21', '26'}


### 2.2 - Debugging 

In [18]:
# Here we will iterate through the Cerdit transaction and check if the MIDIDs from the GWID dict are all in these rows 
import re

# Dictionary to store MID IDs
midid_dict = {}

for row in credit_data.itertuples(index=True):
    description = str(row.clean_description).strip()  # Original description (used for MID IDs)

    midid = None  # Initialize MID ID variable

    # Extract MID ID using original description
    text1 = "indid"
    text2 = "indname"
    text3 = "origid"

    text1_position = description.find(text1)
    text2_position1 = description.find(text2)  # Position from original string
    text3_position1 = description.find(text3)

    if text1_position != -1 and text2_position1 != -1 and text1_position < text2_position1:
        midid = description[text1_position + len(text1): text2_position1].strip()
    elif text1_position != -1 and text3_position1 != -1 and text1_position < text3_position1:
        midid = description[text1_position + len(text1): text3_position1].strip()

    # Store the extracted MID ID
    if midid:
        midid_dict[row.Index] = midid  # Store MID ID with its row index

print(f"🔢 Extracted {len(midid_dict)} MID IDs")
print(f"The percentage of rows that don't have a MIDID is: {round((1 - len(midid_dict) / credit_data.shape[0]) * 100, 2)}%")




🔢 Extracted 165 MID IDs
The percentage of rows that don't have a MIDID is: 0.6%


### Now we wil inspect the transactions that do not have a midid


In [19]:
df_no_midid = credit_data[~credit_data.index.isin(midid_dict.keys())]
df_no_midid

,details,posting_date,description,amount,type,balance,check_or_slip,clean_description
0,CREDIT,6/20/2025,Beginning balance as of 06/20/2025,NaN,ACH_CREDIT,,,beginningbalanceasof06202025


In [20]:
credit_data.shape

(166, 8)

## 3 -  Get the credit types 

### Credit Types in Bank Statements (Affiliate Sales)

1. **merchdep** - **Merchant Deposit**: Funds deposited into your account from sales or transactions processed through a merchant account.

2. **mtotdep** - **Merchant Total Deposit**: The total amount of funds deposited from merchant sales or transactions.

3. **m** - **Merchant**: A generic term for a transaction related to merchant sales or processing.

4. **merchchbk** - **Merchant Chargeback**: A reversal of funds due to a customer dispute or chargeback on a merchant transaction.

5. **cmschrgbks** - **Chargebacks**: Refers to chargebacks processed against your account, typically related to disputed transactions.

6. **pymtproc** - **Payment Processing**: Fees or credits related to payment processing services.

7. **cmsrelease** - **Commission Release**: Funds released to you from held or reserved amounts, often related to affiliate commissions.

8. **btotdep** - **Batch Total Deposit**: The total amount deposited from a batch of transactions.

9. **crcddep** - **Credit Card Deposit**: Funds deposited from credit card transactions.

10. **tpresrel** - **Temporary Reserve Release**: Funds released from a temporary reserve or hold account.

11. **merchrrel** - **Merchant Reserve Release**: Funds released from a merchant reserve account, often held for risk management.

12. **crcdchbk** - **Credit Card Chargeback**: A chargeback specifically related to a credit card transaction.

13. **adjustment** - **Adjustment**: A correction or adjustment made to your account, which could be a credit or debit.

14. **merchfees** - **Merchant Fees**: Fees deducted from your account for merchant services, such as transaction fees.

15. **rdr** - **Reserve Drawdown Release**: Funds released from a reserve account that was held as a security measure.

16. **rsrverlsd** - **Reserve Released**: Funds released from a reserve account.

17. **merchadj** - **Merchant Adjustment**: An adjustment made to your merchant account, often related to corrections or disputes.

18. **merchrel** - **Merchant Release**: Funds released from a merchant account, often after a hold period.

19. **refund** - **Refund**: A credit issued to a customer, resulting in a deduction from your account.

20. **achpmt** - **ACH Payment**: A credit or debit related to an Automated Clearing House (ACH) transaction, such as a direct deposit or withdrawal.

In [21]:
import re

count = 0 
transactions_with_credit = {}
unique_credit_charges = {}

# Extract credit charges (from both coentrydescr...sec and des...indid)
for row in credit_data.itertuples(index=True):
    description = str(row.clean_description).strip().lower()  # Lowercase for consistency
    description_no_numbers = re.sub(r'\d+', '', description)  # Remove all numbers

    found_charge = False  # Track if we found a charge for this row

    # 1. Existing logic for credit charge (coentrydescr...sec)
    text1 = "coentrydescr"
    text2 = "sec"
    text1_position = description_no_numbers.find(text1)
    text2_position = description_no_numbers.find(text2)

    if text1_position != -1 and text2_position != -1 and text1_position < text2_position:
        debit_charge = description_no_numbers[text1_position + len(text1):text2_position].strip()
        if debit_charge:
            unique_credit_charges[debit_charge] = 0
            found_charge = True

    # 2. Also extract charge between 'des' and 'indid'
    des_pos = description.find('des')
    indid_pos = description.find('indid')
    if des_pos != -1 and indid_pos != -1 and des_pos < indid_pos:
        charge_between_des_indid = description[des_pos + len('des'):indid_pos].strip(" :")
        if charge_between_des_indid:
            unique_credit_charges[charge_between_des_indid] = 0
            found_charge = True

    # If we found a charge by either method, mark this row as having a credit charge
    if found_charge:
        count += 1 
        transactions_with_credit[row.Index] = 0 

# Create a list of unique credit charges
unique_credit_charges_list = list(unique_credit_charges.keys())

# Print Summary
print(f"These are the number of rows that don't have a credit charge: {credit_data.shape[0] - count}")
print(f"Number of unique credit charges: {len(unique_credit_charges)}")
print(f"These are the unique charges: {unique_credit_charges_list}")

# Filter DataFrame for rows that **do not** have a credit charge
df_no_credit_charge = credit_data[~credit_data.index.isin(transactions_with_credit.keys())]

# Display the DataFrame with missing credit charges
print("\n📋 DataFrame with rows missing a credit charge:")
print(df_no_credit_charge)

print(unique_credit_charges_list)

These are the number of rows that don't have a credit charge: 1
Number of unique credit charges: 17
These are the unique charges: ['mtot', 'merch', 'tpres', 'merchdep', 'crmerchdepsecccd', 'merchrrel', 'crmerchrrelsecccd', 'tpresrel', 'crtpresrelsecccd', 'crcddep', 'crcrcddepsecccd', 'mtotdep', 'crmtotdepsecccd', 'merchchbk', 'crmerchchbksecccd', 'crcdchbk', 'crcrcdchbksecccd']

📋 DataFrame with rows missing a credit charge:
  details posting_date                         description  amount  \
0  CREDIT    6/20/2025  Beginning balance as of 06/20/2025     NaN   

         type balance check_or_slip             clean_description  
0  ACH_CREDIT                        beginningbalanceasof06202025  
['mtot', 'merch', 'tpres', 'merchdep', 'crmerchdepsecccd', 'merchrrel', 'crmerchrrelsecccd', 'tpresrel', 'crtpresrelsecccd', 'crcddep', 'crcrcddepsecccd', 'mtotdep', 'crmtotdepsecccd', 'merchchbk', 'crmerchchbksecccd', 'crcdchbk', 'crcrcdchbksecccd']


In [22]:
for row in credit_data.itertuples(index=True):
    print(row)

Pandas(Index=0, details='CREDIT', posting_date='6/20/2025', description='Beginning balance as of 06/20/2025', amount=nan, type='ACH_CREDIT', balance='', check_or_slip='', clean_description='beginningbalanceasof06202025')
Pandas(Index=1, details='CREDIT', posting_date='6/20/2025', description='BANKCARD DES:MTOT IND ID:422899490017267 IND NAME:DIVERSE GENERIC PRODUC CO ID:6895422899 CCD', amount=504.48, type='ACH_CREDIT', balance='', check_or_slip='', clean_description='bankcarddesmtotindid422899490017267indnamediversegenericproduccoid6895422899ccd')
Pandas(Index=2, details='CREDIT', posting_date='6/20/2025', description='BANKCARD DEP DES:MERCH IND ID:739788746585208 IND NAME:COMPLETEDOGCAREESSENTI CO ID:203050521 CCD', amount=388.47, type='ACH_CREDIT', balance='', check_or_slip='', clean_description='bankcarddepdesmerchindid739788746585208indnamecompletedogcareessenticoid203050521ccd')
Pandas(Index=3, details='CREDIT', posting_date='6/20/2025', description='MERCHANT SERVICE DES:MERCH IN

In [23]:
df_no_credit_charge

,details,posting_date,description,amount,type,balance,check_or_slip,clean_description
0,CREDIT,6/20/2025,Beginning balance as of 06/20/2025,NaN,ACH_CREDIT,,,beginningbalanceasof06202025


In [24]:
credit_data

,details,posting_date,description,amount,type,balance,check_or_slip,clean_description
0,CREDIT,6/20/2025,Beginning balance as of 06/20/2025,NaN,ACH_CREDIT,,,beginningbalanceasof06202025
1,CREDIT,6/20/2025,BANKCARD DES:MTOT IND ID:422899490017267 IND N...,504.48,ACH_CREDIT,,,bankcarddesmtotindid422899490017267indnamedive...
2,CREDIT,6/20/2025,BANKCARD DEP DES:MERCH IND ID:739788746585208 ...,388.47,ACH_CREDIT,,,bankcarddepdesmerchindid739788746585208indname...
3,CREDIT,6/20/2025,MERCHANT SERVICE DES:MERCH IND ID:180668612303...,384.25,ACH_CREDIT,,,merchantservicedesmerchindid180668612303998ind...
4,CREDIT,6/20/2025,MERCHANT SERVICE DES:MERCH IND ID:180619392303...,364.63,ACH_CREDIT,,,merchantservicedesmerchindid180619392303972ind...
...,...,...,...,...,...,...,...,...
221,CREDIT,6/23/2025,ORIG CO NAME:BANKCARD DEP CO ENTRY DESCR:M...,233.81,ACH_CREDIT,,NaN,origconamebankcarddepcoentrydescrmerchdepseccc...
222,CREDIT,6/23/2025,ORIG CO NAME:MERCHANT SERVICE CO ENTRY DESCR:M...,249.65,ACH_CREDIT,,NaN,origconamemerchantservicecoentrydescrmerchdeps...
223,CREDIT,6/23/2025,ORIG CO NAME:BANKCARD DEP CO ENTRY DESCR:M...,319.84,ACH_CREDIT,,NaN,origconamebankcarddepcoentrydescrmerchdepseccc...
224,CREDIT,6/23/2025,ORIG CO NAME:MERCH SERV CO ENTRY DESCR:M...,385.90,ACH_CREDIT,,NaN,origconamemerchservcoentrydescrmerchdepsecccdi...


## 4 - Create the BackBone dictionary

### This is the dictionary that we will be filling as we iterate through the credit data. The dict will be constructed in the following manner:

- **Keys**: The credit transaction types  
- **Values**: Dict  
    - **Dict has as Keys**: `MIDID`  
    - **Dict has as Values**: `0` (for now)  

In [25]:
# Create the Backbone dictionary
BackBoneGWID_dict = {credit: {} for credit in unique_credit_charges_list}

# Populate it with MID IDs from GWID_dict
for credit in BackBoneGWID_dict:
    for gateway, mid in GWID_dict.items():  # GWID_dict should map GatewayID → MIDID
        BackBoneGWID_dict[credit][mid] = 0  # Use MIDID as the key
BackBoneGWID_dict


{'mtot': {'567000000859181': 0,
  '9180621072288880': 0,
  '9180363862288840': 0,
  '9180454822288910': 0,
  '9180941172288810': 0,
  '567000000864041': 0,
  '567000000864025': 0,
  '8739744451333230': 0,
  '8739724272527570': 0,
  '8739780712387270': 0,
  '8739732218289060': 0,
  '8739768915392320': 0,
  '8739740872762050': 0,
  '7620306740163700': 0,
  '7620869500163710': 0,
  '934600000133389': 0,
  '934600000133371': 0,
  '934600000133363': 0,
  '934600000133355': 0,
  '934600000133462': 0,
  '934600000133454': 0,
  '934600000133470': 0,
  '934600000133488': 0,
  '7620411880163880': 0,
  '7620316740163890': 0,
  '7620773430164130': 0,
  '8739769081527640': 0,
  '8739724341320270': 0,
  '8739704648639800': 0,
  '6316058352030800': 0,
  '6316230133044240': 0,
  '6316020841246130': 0,
  '567000000867614': 0,
  '567000000867606': 0,
  '567000000887281': 0,
  '9180412362295790': 0,
  '9180591502297570': 0,
  '9180991582295780': 0,
  '9180402792300490': 0,
  '9180231672299180': 0,
  '918

### 4.1. - Filling the BackBone

In [26]:
import re

# Sort credit charges list by length to avoid partial matches
unique_credit_charges_list = sorted(unique_credit_charges_list, key=len, reverse=True)

index_used_dict = {}
count = 0

# Iterate through credit_data rows
for row in credit_data.itertuples(index=True):
    description = str(row.clean_description).strip()  # Clean description
    description2 = re.sub(r'\d+', '', description)  # Remove numbers

    amount = row.amount  # Capture transaction amount

    # Extract Credit Charge using BOTH methods (like in discovery phase)
    credit_charge = None
    found_charge = False

    # Method 1: Between "coentrydescr" and "sec"
    text1 = "coentrydescr"
    text2 = "sec"
    text1_position = description2.find(text1)
    text2_position = description2.find(text2)

    if text1_position != -1 and text2_position != -1 and text1_position < text2_position:
        credit_charge = description2[text1_position + len(text1):text2_position].strip()
        if credit_charge:
            found_charge = True

    # Method 2: Between "des" and "indid" (if Method 1 didn't work)
    if not found_charge:
        des_pos = description.find('des')
        indid_pos = description.find('indid')
        if des_pos != -1 and indid_pos != -1 and des_pos < indid_pos:
            credit_charge = description[des_pos + len('des'):indid_pos].strip(" :")
            if credit_charge:
                found_charge = True

    # Extract MID ID
    text3 = "indid"
    text4 = "indname"
    text5 = "origid"
    text3_position = description.find(text3)
    text4_position = description.find(text4)
    text5_position = description.find(text5)

    midid = description[text3_position + len(text3):text4_position].strip()

    if text5 in midid: 
        midid = description[text3_position + len(text3):text5_position].strip()
    
    print(f"Credit Charge: {credit_charge}, MID ID: {midid}")

    # ✅ Fix: Ensure that both `credit_charge` and `midid` exist in `BackBoneGWID_dict`
    if credit_charge and credit_charge in BackBoneGWID_dict:
        if midid not in BackBoneGWID_dict[credit_charge]:  
            # Initialize MID ID if missing
            BackBoneGWID_dict[credit_charge][midid] = 0  

        # ✅ Safe Update
        BackBoneGWID_dict[credit_charge][midid] += amount
        count += 1
        index_used_dict[row.Index] = 0
    else:
        if not credit_charge:
            print(f"⚠️ No credit charge found for transaction")
        else:
            print(f"⚠️ Missing Credit Charge in BackBoneGWID_dict: {credit_charge}")

# Print Summary
print(f"🔢 Rows used: {count}")
BackBoneGWID_dict

Credit Charge: None, MID ID: nningbalanceasof0620202
⚠️ No credit charge found for transaction
Credit Charge: mtot, MID ID: 422899490017267
Credit Charge: merch, MID ID: 739788746585208
Credit Charge: merch, MID ID: 180668612303998
Credit Charge: merch, MID ID: 180619392303972
Credit Charge: tpres, MID ID: 567000000870345
Credit Charge: merch, MID ID: 180668612303998
Credit Charge: tpres, MID ID: 567000000870345
Credit Charge: merch, MID ID: 739729764950577
Credit Charge: merch, MID ID: 180142512303931
Credit Charge: merch, MID ID: 739729764950577
Credit Charge: merch, MID ID: 180142512303931
Credit Charge: merch, MID ID: 739764429769568
Credit Charge: merch, MID ID: 739788746585208
Credit Charge: merch, MID ID: 180674002303949
Credit Charge: mtot, MID ID: 422899490017267
Credit Charge: merch, MID ID: 180674002303949
Credit Charge: tpres, MID ID: 567000000870352
Credit Charge: mtot, MID ID: 422899490017268
Credit Charge: mtot, MID ID: 422899490017268
Credit Charge: merchdep, MID ID: 93

{'mtot': {'567000000859181': 0,
  '9180621072288880': 0,
  '9180363862288840': 0,
  '9180454822288910': 0,
  '9180941172288810': 0,
  '567000000864041': 0,
  '567000000864025': 0,
  '8739744451333230': 0,
  '8739724272527570': 0,
  '8739780712387270': 0,
  '8739732218289060': 0,
  '8739768915392320': 0,
  '8739740872762050': 0,
  '7620306740163700': 0,
  '7620869500163710': 0,
  '934600000133389': 0,
  '934600000133371': 0,
  '934600000133363': 0,
  '934600000133355': 0,
  '934600000133462': 0,
  '934600000133454': 0,
  '934600000133470': 0,
  '934600000133488': 0,
  '7620411880163880': 0,
  '7620316740163890': 0,
  '7620773430164130': 0,
  '8739769081527640': 0,
  '8739724341320270': 0,
  '8739704648639800': 0,
  '6316058352030800': 0,
  '6316230133044240': 0,
  '6316020841246130': 0,
  '567000000867614': 0,
  '567000000867606': 0,
  '567000000887281': 0,
  '9180412362295790': 0,
  '9180591502297570': 0,
  '9180991582295780': 0,
  '9180402792300490': 0,
  '9180231672299180': 0,
  '918

Now we will convert it to a dataframe

In [27]:
#Now we will turn it into a dataframe
df_backbone = pd.DataFrame.from_dict(BackBoneGWID_dict, orient='index')
df_backbone = df_backbone.fillna(0)


# Create a total row
df_backbone.loc["Total"] = df_backbone.sum()
# Drio all columns that have a total of 0
df_backbone = df_backbone.loc[:, df_backbone.loc["Total"] != 0]
print("Are the Totals matching?:", round(df_backbone.loc["Total"].sum(), 3) == round(credit_data["amount"].sum(), 3))
df_backbone

Are the Totals matching?: True


,567000000859181,567000000864041,567000000864025,934600000133389,934600000133371,934600000133363,934600000133355,934600000133462,934600000133454,934600000133470,...,739704648639803,739769081527645,316058352030807,316020841246139,316230133044247,422899490016955,422899490016884,422899490016860,422899490016887,422899490016885
mtot,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0
merch,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0
tpres,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0
merchdep,0.00,0.00,0.00,0.92,0.00,0.00,0.00,12.91,0.00,0.00,...,319.84,427.94,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0
crmerchdepsecccd,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0
merchrrel,0.00,0.00,0.00,1159.18,241.82,367.16,826.14,197.20,967.73,388.13,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0
crmerchrrelsecccd,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0
tpresrel,63.74,648.07,1280.96,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0
crtpresrelsecccd,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0
crcddep,51.10,0.00,74.60,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,242.63,275.61,436.17,0.00,0.00,0.00,0.00,0.0


**Finale - Map the GWID  to change the column names**
   - These will be the steops taken to conclude the project:
     1. Create a for loop that iterates over the column names.✅
     2. Dynamic threshold for small ids and larger ids.✅
     3. Check 1: Check if the midid from the column matches exactlty with that of the GWID_dict midid. ✅
     4. Check 2: If it doesnt find one we use the rapidfuzz✅
     5. If the result is over 85% we substitite the current column name by that of the BWID dict✅
     6. If the result is below 85% add this id to a list of unmatched ids. ✅
     7. Debugging: 1) Print Totals 2) Check how many colum names match with the dict ✅
     8. In the end we must have a df with the same totals, but updated column names that match the middis from the dictionary in question, and with the unmatched ones there still (for now).✅
     

In [28]:
from rapidfuzz import process, fuzz


In [29]:
# Prepare list of known MID IDs (from GWID_dict values)
client_mids_cleaned = {str(mid).strip() for mid in GWID_dict.values()}  # Ensure clean strings

# Here we initiate the list called no_match, which are bank midids were there was no strong prediciton 
no_match = []
# Step 1: create the loop that will iterate over the column names
for bank_mid in df_backbone.columns:

    # Step 2: Determine dynamic threshold based on MID length
    mid_length = len(bank_mid)
    threshold = 95 if mid_length <= 10 else 85  # Stricter for short MIDs, relaxed for long MIDs

    # Step3: Check for exact substring matches first (before fuzzy logic)
    substring_match = next((mid for mid in client_mids_cleaned if bank_mid in mid or mid in bank_mid), None)

    #If exact match found, rename the column
    if substring_match:
        df_backbone.rename(columns={bank_mid: substring_match}, inplace=True)
    
    # Step 4: Check 2: If it doesnt find a perfect match use the rapidfuzz
    else:
        best_match, score, _ = process.extractOne(bank_mid, client_mids_cleaned, scorer=fuzz.partial_ratio)
        if score >= threshold:
            # Step 5: IF the confidence is above the threshold, rename the column   
            df_backbone.rename(columns={bank_mid: best_match}, inplace=True)
        
        # Step 6: If the confidence is low, we will append the mid to the no_match list
        else:
            no_match.append(bank_mid)
            print(f"⚠️ No match found for MID: {bank_mid}, score was {score}")

# Step 7: Debbuging
# Debug 1 - Check the number of columns in the DataFrame are midids in GWID_dict
matching_columns_count = sum(col in client_mids_cleaned for col in df_backbone.columns)
print(f"ℹ️ Number of columns in DataFrame that are mids in BWID_dict: {matching_columns_count}")
# Debug 2 - Check the totals
print("Are the Totals matching?:", round(df_backbone.loc["Total"].sum(), 3) == round(credit_data["amount"].sum(), 3))

# Tsep 8: Print the DataFrame
df_backbone

ℹ️ Number of columns in DataFrame that are mids in BWID_dict: 72
Are the Totals matching?: True


,567000000859181,567000000864041,567000000864025,934600000133389,934600000133371,934600000133363,934600000133355,934600000133462,934600000133454,934600000133470,...,8739704648639800,8739769081527640,6316058352030800,6316020841246130,6316230133044240,4228994900169550,4228994900168840,4228994900168600,4228994900168870,4228994900168850
mtot,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0
merch,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0
tpres,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0
merchdep,0.00,0.00,0.00,0.92,0.00,0.00,0.00,12.91,0.00,0.00,...,319.84,427.94,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0
crmerchdepsecccd,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0
merchrrel,0.00,0.00,0.00,1159.18,241.82,367.16,826.14,197.20,967.73,388.13,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0
crmerchrrelsecccd,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0
tpresrel,63.74,648.07,1280.96,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0
crtpresrelsecccd,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0
crcddep,51.10,0.00,74.60,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,242.63,275.61,436.17,0.00,0.00,0.00,0.00,0.0


## Now we will map this into gateway ids:
    1) Map to gatewayid
    2) Convert to unique columns
    3) Split into gateway matched and non gateway matched
    

### Step 1 - Map tp GatewayID

In [30]:
# Start by inverting the original GWID_dict as we will use this to iterate over the columns
MID_GWID_dict = {v: k for k, v in GWID_dict.items()}

# Now we map it to convert column names to the original GWID 
df_backbone.columns = [MID_GWID_dict.get(col, col) for col in df_backbone.columns]


### Step 2 - Convert to unique columns

In [31]:
# Make sure the columns names are unique
df_backbone = df_backbone.T.groupby(level=0).sum().T

### Step 3 - Split into gateway ids matches and no mathces

In [32]:
# Ensure GWID_dict keys are strings
valid_gateway_ids = set(map(str, GWID_dict.keys()))  # Extract and standardize the GWID keys

# Filter the DataFrame to keep only columns that are valid Gateway IDs
gwid_df = df_backbone.loc[:, df_backbone.columns.isin(valid_gateway_ids)]

print(f"\n✅ Filtered DataFrame to include only valid Gateway ID columns ({len(df_backbone.columns)} columns remaining)")

# Sort gwid_df ascending based on column name (they are numbers)
gwid_df = gwid_df.reindex(sorted(gwid_df.columns, key=int), axis=1)

# Remove rows if all values are 0
gwid_df = gwid_df.loc[(gwid_df != 0).any(axis=1)]
gwid_df


✅ Filtered DataFrame to include only valid Gateway ID columns (72 columns remaining)


,2,3,4,5,6,7,8,9,10,11,...,69,70,73,74,75,76,77,81,86,87
mtot,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.0,0.00,0.00,...,0.00,0.0,0.00,0.00,0.0,0.00,0.00,0.00,52.67,597.7
merch,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.0,0.00,0.00,...,0.00,0.0,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.0
tpres,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.0,0.00,0.00,...,0.00,0.0,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.0
merchdep,0.00,330.14,72.45,936.0,300.93,0.00,0.00,84.2,406.86,161.15,...,476.87,0.0,0.00,0.00,0.0,0.00,215.94,0.00,0.00,0.0
merchrrel,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.0,0.00,0.00,...,0.00,421.5,0.00,0.00,762.3,31.48,893.95,878.91,0.00,0.0
tpresrel,63.74,0.00,0.00,0.0,0.00,648.07,1280.96,0.0,0.00,0.00,...,0.00,0.0,524.28,0.00,0.0,0.00,0.00,0.00,0.00,0.0
crcddep,51.10,0.00,0.00,0.0,0.00,0.00,74.60,0.0,0.00,0.00,...,0.00,0.0,41.54,0.00,0.0,0.00,0.00,0.00,0.00,0.0
mtotdep,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.0,0.00,0.00,...,0.00,0.0,0.00,63.71,0.0,0.00,0.00,0.00,0.00,0.0
merchchbk,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.0,0.00,0.00,...,0.00,0.0,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.0
crcdchbk,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.0,0.00,0.00,...,0.00,0.0,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.0


In [33]:
# Extract the column names that were NOT in GWID_dict keys
missed_columns = [col for col in df_backbone.columns if col not in valid_gateway_ids]

# Create `missed_df` with only those columns
missed_df = df_backbone.loc[:, missed_columns]

# Remove rows where all values are 0
missed_df = missed_df.loc[(missed_df != 0).any(axis=1)]

missed_df


""


## Step 3 -  Now we create the credit transaction - midid dataframe but for Denaut
- **Step1**: Create the start dict, where the keys are the credit names, values are another dict. This other dict, has as keys the midids and as values the totals for that midid.~
- **Step 1**: Iteate over the denaut dataframe and populate the start_dict

## Debugging

## Step 3 - Convert to credit transaction - midid column

## Now that we have the right dict with Denuat MIDIDs and their respective gwids we can map it to denaut gwid dataframe

# Hand-in 

### 1) Final GWID_df, index as credit charge charges and GWID as columns

In [34]:
# Remove existing CreditName column if it exists
if "CreditName" in gwid_df.columns:
    gwid_df = gwid_df.drop(columns=["CreditName"])

# Map internal codes (index) to readable names
credit_desc_dict = {
    "merchdep": "Merchant Deposit",
    "tpresrel": "Merchant Deposit",
    "merchrrel": "Merchant Release",
    "adjustment": "Adjustment",
    "crcddep": "Merchant Deposit",
    "merchchbk": "Chargeback Refunded",
    "ach credit": "Ach Credit"
}

# Apply dictionary mapping; fallback to "no description"
gwid_df["CreditName"] = gwid_df.index.to_series().map(credit_desc_dict).fillna("no description")

# Calculate Total per row (excluding strings like CreditName)
gwid_df["Total"] = gwid_df.select_dtypes(include='number').sum(axis=1)

# Add Total row at the bottom
gwid_df.loc["Total"] = gwid_df.drop(index="Total", errors="ignore").select_dtypes(include="number").sum()
gwid_df.loc["Total", "CreditName"] = "Total"

# make sure that if Credit Name in the gwid_df that it has to be in the first column
gwid_df = gwid_df[["CreditName"] + [col for col in gwid_df.columns if col != "CreditName"]]
print("✅ Final GWID sheet generated!")
gwid_df



✅ Final GWID sheet generated!


,CreditName,2,3,4,5,6,7,8,9,10,...,70,73,74,75,76,77,81,86,87,Total
mtot,no description,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.0,0.00,...,0.0,0.00,0.00,0.0,0.00,0.00,0.00,52.67,597.7,650.37
merch,no description,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.0,0.00,...,0.0,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.0,2733.85
tpres,no description,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.0,0.00,...,0.0,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.0,695.32
merchdep,Merchant Deposit,0.00,330.14,72.45,936.0,300.93,0.00,0.00,84.2,406.86,...,0.0,0.00,0.00,0.0,0.00,215.94,0.00,0.00,0.0,15235.01
merchrrel,Merchant Release,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.0,0.00,...,421.5,0.00,0.00,762.3,31.48,893.95,878.91,0.00,0.0,9770.28
tpresrel,Merchant Deposit,63.74,0.00,0.00,0.0,0.00,648.07,1280.96,0.0,0.00,...,0.0,524.28,0.00,0.0,0.00,0.00,0.00,0.00,0.0,3783.83
crcddep,Merchant Deposit,51.10,0.00,0.00,0.0,0.00,0.00,74.60,0.0,0.00,...,0.0,41.54,0.00,0.0,0.00,0.00,0.00,0.00,0.0,1135.62
mtotdep,no description,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.0,0.00,...,0.0,0.00,63.71,0.0,0.00,0.00,0.00,0.00,0.0,953.81
merchchbk,Chargeback Refunded,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.0,0.00,...,0.0,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.0,47.84
crcdchbk,no description,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.0,0.00,...,0.0,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.0,98.89


## 2) Final Missed df

In [35]:
missed_df = missed_df.drop(columns=["CreditName"], errors='ignore')

# Create list to fill in here
credit_name_list = [
    None,
    None
]

missed_df = missed_df.rename(index={"m": "PlushPlay"})
# Ensure `credit_name_list` has enough values for all rows
while len(credit_name_list) - 1 < len(missed_df):
    credit_name_list.append("None")  # Add placeholder values

# Insert "CreditName" column (excluding the title itself)
missed_df.insert(0, "CreditName", credit_name_list[1:])

print("\n✅ Successfully added 'CreditName' column!")

# Add columnn that checks total by row 
missed_df["Total"] = missed_df.select_dtypes(include=['number']).sum(axis=1)

# Corrert the Total Total cell
missed_df.loc["Total", "Total"] = None

# Drop teh credit name column 
missed_df = missed_df.drop(columns=["CreditName"], errors='ignore')

# drop the Total row 
missed_df = missed_df.drop("Total")

# Create a Total row which is teh sum by column 
missed_df.loc["Total"] = missed_df.select_dtypes(include='number').sum()

# Drop the Total column 
missed_df = missed_df.drop(columns="Total")

# In the cell where column is Dec and row is Total make sure that teh value is None
missed_df.loc["Total", "Desc"] = "Total"
missed_df


✅ Successfully added 'CreditName' column!


,Desc
0,NaN
Total,Total


In [36]:
gwid_df.loc["Total", "Desc"] = "Total"

C:\Users\mmsou\AppData\Local\Temp\ipykernel_36028\3603329035.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gwid_df.loc["Total", "Desc"] = "Total"


## 3) Processor Dataframe

In [37]:
import pandas as pd
gwid_copy = gwid_df.copy()
# Assuming gwid_df and processor_gw_dict are already defined

# Step 1: Create a reverse mapping from gateway ID to processor
gwid_to_processor = {gwid: processor for processor, gwids in processor_gw_dict.items() for gwid in gwids}

# Step 2: Temporarily remove the CreditName column
credit_name_column = gwid_copy.pop('CreditName')

# Step 3: Rename gwid_copy columns using the processor mapping
processor_df = gwid_copy.rename(columns=lambda x: gwid_to_processor.get(str(x), str(x)))

# Step 4: Merge duplicate columns (now using processor names)
processor_df = processor_df.T.groupby(processor_df.columns, axis=0).sum().T

# Step 5: Sort columns alphabetically (now all names are unique)
processor_df = processor_df.reindex(sorted(processor_df.columns), axis=1)

# Step 6: Reinsert the CreditName column as the first column
processor_df.insert(0, 'CreditName', credit_name_column)

# Make sure the intersection between column CreditName and Total is "Total"
processor_df.loc["Total", "CreditName"] = "Total"

# Drop the Desc columns 
processor_df = processor_df.drop(columns=["Desc"], errors='ignore')

# Display the updated DataFrame
processor_df


C:\Users\mmsou\AppData\Local\Temp\ipykernel_36028\356822938.py:15: FutureWarning: The 'axis' keyword in DataFrame.groupby is deprecated and will be removed in a future version.
  processor_df = processor_df.T.groupby(processor_df.columns, axis=0).sum().T


,CreditName,APPS(SYNOVUS)(PLUSH PAY),Esquire (Netevia),Evolve (PayArc),PAYSAFE (BBVA)(PLUSH PAY),PAYSAFE(BBVA)(PLUSH PAY),PAYSAFE(PNC)(PLUSH PAY),QUANTUM (CBSL) (JORDAN),Signapay (Esquire)(Travis),Synovus,Total
mtot,no description,0.0,0.0,0.0,650.37,0.0,0.0,0.0,0.0,0.0,650.37
merch,no description,0.0,1612.01,0.0,0.0,0.0,0.0,0.0,0.0,1121.84,2733.85
tpres,no description,0.0,0.0,695.32,0.0,0.0,0.0,0.0,0.0,0.0,695.32
merchdep,Merchant Deposit,0.0,9764.44,0.0,0.0,0.0,0.0,245.62,3069.2,2155.75,15235.01
merchrrel,Merchant Release,0.0,0.0,0.0,0.0,0.0,0.0,9770.28,0.0,0.0,9770.28
tpresrel,Merchant Deposit,0.0,0.0,3783.83,0.0,0.0,0.0,0.0,0.0,0.0,3783.83
crcddep,Merchant Deposit,954.41,0.0,181.21,0.0,0.0,0.0,0.0,0.0,0.0,1135.62
mtotdep,no description,0.0,0.0,0.0,412.04,493.99,47.78,0.0,0.0,0.0,953.81
merchchbk,Chargeback Refunded,0.0,47.84,0.0,0.0,0.0,0.0,0.0,0.0,0.0,47.84
crcdchbk,no description,0.0,0.0,98.89,0.0,0.0,0.0,0.0,0.0,0.0,98.89


## 4) Hand-In



* Step 1: Make sure the Deposited, Y/N column is in there
* Step 2: Create a new dataframe called gwid_summary which is the same as gwid_df removing all rows except for the Total, and also removing the Index and CreditName tables
* Step 3: Iterate over reconciliation dataframe
    * Step 3.1: Extract number from Merchant column
    * Step 3.2: Extract index of ths column
    * Step 3.3: With the extracted number, check the gwid_df_summary sheet, if Total > 0, add Y to Credits column in reconciliation else N 
* Step 4: Check the reconciliation df and see if results are correct

In [38]:
def clean_column_names(df):
    df.columns = (
        df.columns
        .str.strip()  # Remove leading/trailing spaces
        .str.replace(r"[^\w\s]", "", regex=True)  # Remove special characters
        .str.replace(r"\s+", "_", regex=True)  # Replace spaces with underscores
        .str.lower()  # Convert to lowercase
    )
    return df

In [39]:
import glob
import os
import pandas as pd

# Get current notebook directory
folder_path = os.getcwd()  # gets the folder where the notebook is located

# Get a function that will clean all the strings 
def clean_string(text):
    """
    Clean a string by removing special characters, spaces, and numbers, converting to lowercase
    
    Args:
        text (str): The input string to clean
        
    Returns:
        str: Cleaned string with only alphabetic characters in lowercase
    """
    import re
    # Remove special characters, numbers, and spaces, convert to lowercase
    cleaned = re.sub(r'[^a-zA-Z]', '', text).lower()
    return cleaned

def find_deposit_file(folder_path, base_words=["deprec", "export"]):
    # Match any csv/xls/xlsx containing the keywords
    all_files = os.listdir(folder_path)
    matching_files = []
    
    for file in all_files:
        # Only check files with csv or excel extensions
        if file.lower().endswith(('.csv', '.xls', '.xlsx')):
            file_clean = clean_string(file)
            # Check if all required words are in the cleaned filename
            if all(word in file_clean for word in base_words):
                matching_files.append(os.path.join(folder_path, file))
                print(f"Found matching file: {file}")
    
    if not matching_files:
        raise FileNotFoundError(f"No matching files found in {folder_path} containing {base_words}")
        
    # Sort by most recent modified time
    matching_files.sort(key=os.path.getmtime, reverse=True)
    return matching_files[0]  # return the newest one

# Now load it
file_path = find_deposit_file(folder_path)
print(f"📄 Loading file: {file_path}")

# Load the file with previous flexible loader
def load_file(file_path):
    ext = os.path.splitext(file_path)[1].lower()
    if ext == '.csv':
        return pd.read_csv(file_path)
    elif ext in ['.xls', '.xlsx']:
        return pd.read_excel(file_path, engine='openpyxl')
    else:
        raise ValueError(f"Unsupported file type: {ext}")

# Load your DataFrame
deposit_reconciliation_df = load_file(file_path)

Found matching file: DEPREC_EXPORT_1750684702.csv
📄 Loading file: c:\Users\mmsou\Documents\mokum.ai\Goldfinger\DEPREC_EXPORT_1750684702.csv


In [40]:
# First define which columns to keep
columns_to_keep_recon = ["Merchant Group", "Merchant", "Net Revenue"]

# Create a new column for Y/N (filled with empty strings)
deposit_reconciliation_df["Y/N"] = ""

# Now add Y/N to the list of columns to keep
columns_to_keep_with_yn = ["Merchant Group", "Merchant", "Y/N", "Net Revenue"]

# Filter the DataFrame to only include the desired columns in the right order
deposit_reconciliation_df = deposit_reconciliation_df[columns_to_keep_with_yn]

deposit_reconciliation_df

,Merchant Group,Merchant,Y/N,Net Revenue
0,BrightAdvantageLLC,12 - RM_12_PRIORITY_BrightAdvantageLLC_9068_75K,,$-60.99
1,BrightAdvantageLLC,13 - RM_13_PRIORITY_BrightAdvantageLLC_2323_75K,,$202.01
2,BrightAdvantageLLC,14 - RM_14_PRIORITY_BrightAdvantageLLC_2051_75K,,$202.01
3,BrightAdvantageLLC,17 - RM_17_SIGNAPAY_BrightAdvantageLLC_3881_50K,,$648.36
4,BrightAdvantageLLC,18 - RM_18_SIGNAPAY_BrightAdvantageLLC_3899_50K,,$315.62
...,...,...,...,...
74,UniquePlusLLC,42 - RM_42_NETEVIA_UniquePlusLLC_9170_65K,,$34.69
75,UniquePlusLLC,43 - RM_43_NETEVIA_UniquePlusLLC_9162_65K,,$363.46
76,UniquePlusLLC,44 - RM_44_NETEVIA_UniquePlusLLC_9154_65K,,$156.51
77,UniquePlusLLC,71 - HOLD_WM_71_PAYSAFE_UniquePlusLLC_9410_50K,,$-12.89


In [41]:
import glob
import os
import pandas as pd

# Get current notebook directory
folder_path = os.getcwd()  # gets the folder where the notebook is located

# Get a function that will clean all the strings 
def clean_string(text):
    """
    Clean a string by removing special characters, spaces, and numbers, converting to lowercase
    
    Args:
        text (str): The input string to clean
        
    Returns:
        str: Cleaned string with only alphabetic characters in lowercase
    """
    import re
    # Remove special characters, numbers, and spaces, convert to lowercase
    cleaned = re.sub(r'[^a-zA-Z]', '', text).lower()
    return cleaned

def find_deposit_file(folder_path, base_words=["deprec", "export"]):
    # Match any csv/xls/xlsx containing the keywords
    all_files = os.listdir(folder_path)
    matching_files = []
    
    for file in all_files:
        # Only check files with csv or excel extensions
        if file.lower().endswith(('.csv', '.xls', '.xlsx')):
            file_clean = clean_string(file)
            # Check if all required words are in the cleaned filename
            if all(word in file_clean for word in base_words):
                matching_files.append(os.path.join(folder_path, file))
                print(f"Found matching file: {file}")
    
    if not matching_files:
        raise FileNotFoundError(f"No matching files found in {folder_path} containing {base_words}")
        
    # Sort by most recent modified time
    matching_files.sort(key=os.path.getmtime, reverse=True)
    return matching_files[0]  # return the newest one

# Now load it
file_path = find_deposit_file(folder_path)
print(f"📄 Loading file: {file_path}")

# Load the file with previous flexible loader
def load_file(file_path):
    ext = os.path.splitext(file_path)[1].lower()
    if ext == '.csv':
        return pd.read_csv(file_path)
    elif ext in ['.xls', '.xlsx']:
        return pd.read_excel(file_path, engine='openpyxl')
    else:
        raise ValueError(f"Unsupported file type: {ext}")

# Load your DataFrame
deposit_reconciliation_df = load_file(file_path)

# First define which columns to keep
columns_to_keep_recon = ["Merchant Group", "Merchant", "Net Revenue"]

# Filter to just the columns we need
deposit_reconciliation_df = deposit_reconciliation_df[columns_to_keep_recon]

# Create a new column for Y/N (filled with empty strings)
deposit_reconciliation_df["Y/N"] = ""

# Reorder columns to get Y/N in the right place
deposit_reconciliation_df = deposit_reconciliation_df[["Merchant Group", "Merchant", "Y/N", "Net Revenue"]]

# Step 2: Create a new dataframe called gwid_summary which is the same as gwid_df 
# removing all rows except for the Total, and also removing the Index and CreditName tables
total_row = gwid_df.loc["Total"]
gwid_summary = total_row.to_frame().T

# Step 3: Iteration
gwid_in_rec_and_in_gwid_summary = []
gwid_in_rec_but_not_in_gwid_summary = []
count = 0 

# Original columns 
original_columns = deposit_reconciliation_df.columns

# Clean the dataframe column names for better iteration
deposit_reconciliation_df = clean_column_names(deposit_reconciliation_df)

# Add new columns
deposit_reconciliation_df["amount_deposited"] = 0

# Ensure "yn" column exists and is the correct type
deposit_reconciliation_df["yn"] = deposit_reconciliation_df.get("yn", pd.Series(dtype="object"))
deposit_reconciliation_df["amount_deposited"] = deposit_reconciliation_df.get("amount_deposited", pd.Series(dtype="float"))

# Main loop
for row in deposit_reconciliation_df.itertuples(index=True):
    split = row.merchant.split("-")
    gwid = str(split[0]).strip()

    if gwid in gwid_summary.columns:
        gwid_in_rec_and_in_gwid_summary.append(gwid)
        total_value = gwid_summary[gwid].values[0]

        deposit_reconciliation_df.at[row.Index, "amount_deposited"] = float(total_value)
        deposit_reconciliation_df.at[row.Index, "yn"] = "Y" if total_value > 0 else "N"
    else:
        gwid_in_rec_but_not_in_gwid_summary.append(gwid)

# Set missing values safely
deposit_reconciliation_df["yn"] = deposit_reconciliation_df["yn"].fillna("N")
deposit_reconciliation_df["amount_deposited"] = deposit_reconciliation_df["amount_deposited"].fillna(0.0)

# Map the column names
column_map = {
    "merchant_group": "Merchant Group",
    "merchant": "Merchant",
    "yn": "Y/N",
    "net_revenue": "Net Revenue",
    "amount_deposited": "Amount Deposited"
}

deposit_reconciliation_df.rename(columns=column_map, inplace=True)

# make sure that the only columns we have here are the ones we want
deposit_reconciliation_df = deposit_reconciliation_df[["Merchant Group", "Merchant", "Y/N", "Net Revenue", "Amount Deposited"]]
deposit_reconciliation_df.loc[deposit_reconciliation_df["Amount Deposited"] == 0, "Y/N"] = "N"
print()
print("GWID in Rec and GWID summary:")
print(gwid_in_rec_and_in_gwid_summary)
print()
print("GWID in Rec but not in GWID summary:")
print(gwid_in_rec_but_not_in_gwid_summary)

deposit_reconciliation_df.head(50)

Found matching file: DEPREC_EXPORT_1750684702.csv
📄 Loading file: c:\Users\mmsou\Documents\mokum.ai\Goldfinger\DEPREC_EXPORT_1750684702.csv

GWID in Rec and GWID summary:
['13', '14', '17', '18', '21', '22', '23', '24', '34', '35', '37', '39', '65', '7', '74', '8', '40', '54', '59', '60', '61', '62', '63', '64', '67', '68', '69', '75', '76', '77', '81', '48', '49', '50', '51', '52', '55', '56', '57', '58', '86', '87', '10', '11', '2', '3', '31', '32', '33', '4', '45', '46', '47', '5', '6', '66', '70', '73', '9', '19', '25', '26', '27', '28', '29', '30', '36', '38', '41', '42', '43', '44']

GWID in Rec but not in GWID summary:
['12', '53', '78', '90', '15', '71', '72']


C:\Users\mmsou\AppData\Local\Temp\ipykernel_36028\2881146950.py:106: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '177.76' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  deposit_reconciliation_df.at[row.Index, "amount_deposited"] = float(total_value)


,Merchant Group,Merchant,Y/N,Net Revenue,Amount Deposited
0,BrightAdvantageLLC,12 - RM_12_PRIORITY_BrightAdvantageLLC_9068_75K,N,$-60.99,0.00
1,BrightAdvantageLLC,13 - RM_13_PRIORITY_BrightAdvantageLLC_2323_75K,Y,$202.01,177.76
2,BrightAdvantageLLC,14 - RM_14_PRIORITY_BrightAdvantageLLC_2051_75K,Y,$202.01,79.40
3,BrightAdvantageLLC,17 - RM_17_SIGNAPAY_BrightAdvantageLLC_3881_50K,Y,$648.36,473.14
4,BrightAdvantageLLC,18 - RM_18_SIGNAPAY_BrightAdvantageLLC_3899_50K,Y,$315.62,430.90
5,BrightAdvantageLLC,21 - RM_21_QUANTUM_BrightAdvantageLLC_3462_25K,Y,$117.84,210.11
6,BrightAdvantageLLC,22 - RM_22_QUANTUM_BrightAdvantageLLC_3454_25K,Y,$479.47,967.73
7,BrightAdvantageLLC,23 - RM_23_QUANTUM_BrightAdvantageLLC_3470_25K,Y,$165.68,388.13
8,BrightAdvantageLLC,24 - RM_24_QUANTUM_BrightAdvantageLLC_3488_25K,Y,$370.54,1050.39
9,BrightAdvantageLLC,34 - RM_34_NETEVIA_BrightAdvantageLLC_5798_55K,Y,$71.28,625.44


### Message spit out:
- 1) Get the cleaned dataframes, so where we have date and where remove the Unnamed column. ✅
- 2) Concat the two into a df called message_p1_df.
- 3) Check if we have two unique dates
- 4) We will be using the Date column to make sure that we only have two dates, this will be necessary to spit out the message.
- 5) Get the correct GWID - Processor pair dictionary
- 6) Message creation:
    - 1) The Repeat list includes MIDs that haven’t deposited on both the current and previous day, which usually indicates a potential issue if the MID has been batching but hasn’t deposited for two consecutive days.
    - 2) The New list includes MIDs that have only missed depositing on the current day.
    - 3) Deposited: all the Ys
    - 4) Not edeposit with revenue negative: self explanatory
- 7) Remove the columns with the latest date, leaving the master df clean for the next callling of P1


In [42]:
import datetime
# Get today's date
today = datetime.date.today()

In [43]:
# Import the xlsx called Master_Reconciliation
message_p1_df = pd.read_excel("Master_Reconciliation_GF.xlsx", engine='openpyxl')


message_p1_df

,Date,Merchant Group,Merchant,Y/N,Net Revenue,Amount Deposited,gwid
0,2025-06-24,BrightAdvantageLLC,12 - RM_12_PRIORITY_BrightAdvantageLLC_9068_75K,N,117.84,0.00,12
1,2025-06-24,BrightAdvantageLLC,13 - RM_13_PRIORITY_BrightAdvantageLLC_2323_75K,Y,103.12,109.63,13
2,2025-06-24,BrightAdvantageLLC,14 - RM_14_PRIORITY_BrightAdvantageLLC_2051_75K,Y,202.01,324.76,14
3,2025-06-24,BrightAdvantageLLC,17 - RM_17_SIGNAPAY_BrightAdvantageLLC_3881_50K,Y,332.74,1475.13,17
4,2025-06-24,BrightAdvantageLLC,18 - RM_18_SIGNAPAY_BrightAdvantageLLC_3899_50K,Y,6.06,1093.42,18
...,...,...,...,...,...,...,...
74,2025-06-24,UniquePlusLLC,42 - RM_42_NETEVIA_UniquePlusLLC_9170_65K,Y,213.52,748.72,42
75,2025-06-24,UniquePlusLLC,43 - RM_43_NETEVIA_UniquePlusLLC_9162_65K,Y,245.62,327.59,43
76,2025-06-24,UniquePlusLLC,44 - RM_44_NETEVIA_UniquePlusLLC_9154_65K,Y,556.20,644.74,44
77,2025-06-24,UniquePlusLLC,71 - HOLD_WM_71_PAYSAFE_UniquePlusLLC_9410_50K,N,0.00,0.00,71


In [44]:
deposit_reconciliation_df_copy = deposit_reconciliation_df.copy()
deposit_reconciliation_df_copy

,Merchant Group,Merchant,Y/N,Net Revenue,Amount Deposited
0,BrightAdvantageLLC,12 - RM_12_PRIORITY_BrightAdvantageLLC_9068_75K,N,$-60.99,0.00
1,BrightAdvantageLLC,13 - RM_13_PRIORITY_BrightAdvantageLLC_2323_75K,Y,$202.01,177.76
2,BrightAdvantageLLC,14 - RM_14_PRIORITY_BrightAdvantageLLC_2051_75K,Y,$202.01,79.40
3,BrightAdvantageLLC,17 - RM_17_SIGNAPAY_BrightAdvantageLLC_3881_50K,Y,$648.36,473.14
4,BrightAdvantageLLC,18 - RM_18_SIGNAPAY_BrightAdvantageLLC_3899_50K,Y,$315.62,430.90
...,...,...,...,...,...
74,UniquePlusLLC,42 - RM_42_NETEVIA_UniquePlusLLC_9170_65K,Y,$34.69,637.14
75,UniquePlusLLC,43 - RM_43_NETEVIA_UniquePlusLLC_9162_65K,Y,$363.46,490.48
76,UniquePlusLLC,44 - RM_44_NETEVIA_UniquePlusLLC_9154_65K,Y,$156.51,724.85
77,UniquePlusLLC,71 - HOLD_WM_71_PAYSAFE_UniquePlusLLC_9410_50K,N,$-12.89,0.00


In [45]:
# Put the Date column in position 0  of the dataframe
message_p1_df.insert(0, "Date", message_p1_df.pop("Date"))
message_p1_df.head()

# Create a new column called Date
today = datetime.date.today()
deposit_reconciliation_df_copy["Date"] = pd.to_datetime(today)

# Insert in position 1
deposit_reconciliation_df_copy.insert(0, "Date", deposit_reconciliation_df_copy.pop("Date"))

# Create a new column called GWID which just has the gwid in it
deposit_reconciliation_df_copy["gwid"] = deposit_reconciliation_df_copy["Merchant"].str.split("-").str[0]

# Remove all spaces from the gwid
deposit_reconciliation_df_copy["gwid"] = deposit_reconciliation_df_copy["gwid"].str.replace(" ", "")

# Convert the gwid to int
deposit_reconciliation_df_copy["gwid"] = deposit_reconciliation_df_copy["gwid"].astype(int)


# Concatenate deposit_reconciliation_df_copy with message_p1_df, using the column format from message_p1_df
correct_columns = message_p1_df.columns
# deposit_reconciliation_df_copy = deposit_reconciliation_df_copy[correct_columns]
# deposit_reconciliation_df_copy.head()
message_p1_df.head()


# Drop the column Unnamed: 0 from deposit_reconciliation_df_copy and message_p1_df if it exists
if "Unnamed: 0" in deposit_reconciliation_df_copy.columns:
    deposit_reconciliation_df_copy.drop(columns=["Unnamed: 0"], inplace=True)
if "Unnamed: 0" in message_p1_df.columns:
    message_p1_df.drop(columns=["Unnamed: 0"], inplace=True)
message_p1_df

,Date,Merchant Group,Merchant,Y/N,Net Revenue,Amount Deposited,gwid
0,2025-06-24,BrightAdvantageLLC,12 - RM_12_PRIORITY_BrightAdvantageLLC_9068_75K,N,117.84,0.00,12
1,2025-06-24,BrightAdvantageLLC,13 - RM_13_PRIORITY_BrightAdvantageLLC_2323_75K,Y,103.12,109.63,13
2,2025-06-24,BrightAdvantageLLC,14 - RM_14_PRIORITY_BrightAdvantageLLC_2051_75K,Y,202.01,324.76,14
3,2025-06-24,BrightAdvantageLLC,17 - RM_17_SIGNAPAY_BrightAdvantageLLC_3881_50K,Y,332.74,1475.13,17
4,2025-06-24,BrightAdvantageLLC,18 - RM_18_SIGNAPAY_BrightAdvantageLLC_3899_50K,Y,6.06,1093.42,18
...,...,...,...,...,...,...,...
74,2025-06-24,UniquePlusLLC,42 - RM_42_NETEVIA_UniquePlusLLC_9170_65K,Y,213.52,748.72,42
75,2025-06-24,UniquePlusLLC,43 - RM_43_NETEVIA_UniquePlusLLC_9162_65K,Y,245.62,327.59,43
76,2025-06-24,UniquePlusLLC,44 - RM_44_NETEVIA_UniquePlusLLC_9154_65K,Y,556.20,644.74,44
77,2025-06-24,UniquePlusLLC,71 - HOLD_WM_71_PAYSAFE_UniquePlusLLC_9410_50K,N,0.00,0.00,71


In [46]:
# Clean Net Revenue in deposit_reconciliation_df_copy by removing the $ sign and converting to float
def clean_currency_column(series):
    return (
        series.astype(str)
        .str.replace("$", "", regex=False)
        .str.replace(",", "", regex=False)
        .str.replace("(", "-", regex=False)
        .str.replace(")", "", regex=False)
        .str.strip()
        .astype(float)
    )

# If the Net Revenue column has $ or () in it, clean it we call the function clean_currency_column
if deposit_reconciliation_df_copy["Net Revenue"].dtype == "object":
    deposit_reconciliation_df_copy["Net Revenue"] = clean_currency_column(deposit_reconciliation_df_copy["Net Revenue"])


In [47]:
# STEP 2: Conact the two

# Conact master and deposit_reconciliation_df_copy, master goes above deposit_reconciliation_df_copy
# Concatenate master_reconciliation_df on top of deposit_reconciliation_df_copy
message_p1_df = pd.concat([message_p1_df, deposit_reconciliation_df_copy], ignore_index=True)

# Display the resulting DataFrame
if message_p1_df.shape[0] == ((message_p1_df.shape[0] + deposit_reconciliation_df_copy.shape[0]) / 2):
    print("Concatenation successful!")
else:
    print("Concatenation failed!")

# Step 3: Check if we have two distinct values in Date
unique_dates = message_p1_df["Date"].unique()
if len(unique_dates) == 2:
    print("Two distinct dates found!")

# Conver the data type of Date to just date
message_p1_df["Date"] = pd.to_datetime(message_p1_df["Date"]).dt.date

# Get the smalles date
smallest_date = message_p1_df["Date"].min()
largest_date = message_p1_df["Date"].max()

print("Largest date:", largest_date)
print("Smallest date:", smallest_date)

# Step 5 - Get the correct GWID - Processor name pair
gwid_mapping_df = pd.read_excel("GWID_local_GF.xlsx", engine='openpyxl')

# Make sure that columns dont have any trailing sapces in them
gwid_mapping_df.columns = gwid_mapping_df.columns.str.strip()

gwid_dict = {}
for row in gwid_mapping_df.itertuples():
    gwid = row.GWID
    processor_name = row.Processor
    gwid_dict[gwid] = processor_name

# Step 6 - Iteration and message creation

message = ""


message += f"Report for today ({today})\n"

# Get the dataframes with the min date and one with the max date
message_p1_df_min = message_p1_df[message_p1_df["Date"] == smallest_date].copy()
message_p1_df_max = message_p1_df[message_p1_df["Date"] == largest_date].copy()

#-------------------------------------------------------  STEP 1 - Repeat part of the message  --------------------------------------------------
# Create a new column called GWID which just has the gwid in it
message_p1_df["gwid"] = message_p1_df["Merchant"].str.split("-").str[0]
# Remove all spaces from the gwid
message_p1_df["gwid"] = message_p1_df["gwid"].str.replace(" ", "")
# Convert the gwid to int
message_p1_df["gwid"] = message_p1_df["gwid"].astype(int)
message_p1_df.head()
# Get the dataframes with the min date and one with the max date
message_p1_df_min = message_p1_df[message_p1_df["Date"] == smallest_date]
message_p1_df_max = message_p1_df[message_p1_df["Date"] == largest_date]
# Group by gateway_id and aggregate the 'Y-M' column
repeats = message_p1_df.groupby('gwid')['Y/N'].apply(lambda x: (x == 'N').sum())

# Step 2: Filter gateway IDs that have 'N' for both dates (i.e., count == 2)
no_deposit_ids_repeat = repeats[repeats == 2].index.tolist()

# check if all no_deposit_ids are in the message_p1_df, just give me a True or False
print(all(gwid in message_p1_df["gwid"].values for gwid in no_deposit_ids_repeat))
message += f"Repeat: \n"
for gwid in no_deposit_ids_repeat:
    if gwid in gwid_dict:
        message += f"{gwid} - {gwid_dict[gwid]} \n"

#-------------------------------------------------------  STEP 2 - New part of the message  --------------------------------------------------

# Filter gwid that are 'Y' in message_p1_df_min and 'N' in message_p1_df_max
gwids_yesterday = message_p1_df_min[message_p1_df_min['Y/N'] == 'Y']['gwid']
gwids_today = message_p1_df_max[message_p1_df_max['Y/N'] == 'N']['gwid']

# Find gwid values that are in both sets
dep_yesterday_nodep_today = gwids_yesterday[gwids_yesterday.isin(gwids_today)].tolist()

# Add to the message
message += f"New: \n"
for gwid in dep_yesterday_nodep_today:
    if gwid in gwid_dict:
        message += f"{gwid} - {gwid_dict[gwid]} \n"

print(dep_yesterday_nodep_today)

"""
#-------------------------------------------------------  STEP 3 - From yesterday deposited part of the message  --------------------------------------------------
# Get the gwid from yesterday that have a Y in them
old = message_p1_df_max.groupby('gwid')['Y/N'].apply(lambda x: (x == 'Y').sum())
deposit_ids_today = old[old == 1].index.tolist()

message += f"From yesterday: \n"
message += f"Deposited: \n"
for gwid in deposit_ids_today:
    if gwid in gwid_dict:
        message += f"{gwid} - {gwid_dict[gwid]} \n"


#-------------------------------------------------------  STEP 4 - Not deposited with revenue negatitve today deposited part of the message  --------------------------------------------------
# Check if the Net Revenue column contains any non-numeric values
if not pd.to_numeric(message_p1_df_min["Net Revenue"], errors='coerce').notna().all():
    # Make the Net Revenue column into a string
    message_p1_df_min["Net Revenue"] = message_p1_df_min["Net Revenue"].astype(str)

    # Remove all special characters and spaces from the Net Revenue column in message_p1_df_min
    message_p1_df_min["Net Revenue"] = message_p1_df_min["Net Revenue"].str.replace(r'[^0-9.-]', '', regex=True)

    # Remove all spaces from the Net Revenue column in message_p1_df_min
    message_p1_df_min["Net Revenue"] = message_p1_df_min["Net Revenue"].str.replace(" ", "")

    # Turn the Net Revenue column into a float in message_p1_df_min
    message_p1_df_min["Net Revenue"] = message_p1_df_min["Net Revenue"].astype(float)

# Create a new column called is_negative_rev where the Net Revenue is less than 0 we have True and False otherwise
message_p1_df_min["is_negative_rev"] = message_p1_df_min["Net Revenue"] < 0

message_p1_df_min.head()

# Filter rows where is_negative_rev is True and y_n is 'N'
negative_revenue_no_deposit = message_p1_df_min[(message_p1_df_min["is_negative_rev"] == True) & (message_p1_df_min["Y/N"] == 'N')]

# Extract the gwid values from the filtered rows
negative_revenue_no_deposit_ids = negative_revenue_no_deposit["gwid"].tolist()

# Add to the message
message += f"Not deposited with revenue in negative: \n"
for gwid in negative_revenue_no_deposit_ids:
    if gwid in gwid_dict:
        message += f"{gwid} - {gwid_dict[gwid]} \n"

"""


Concatenation failed!
Largest date: 2025-06-24
Smallest date: 2025-06-24
True
[53, 74, 54, 76, 2]


'\n#-------------------------------------------------------  STEP 3 - From yesterday deposited part of the message  --------------------------------------------------\n# Get the gwid from yesterday that have a Y in them\nold = message_p1_df_max.groupby(\'gwid\')[\'Y/N\'].apply(lambda x: (x == \'Y\').sum())\ndeposit_ids_today = old[old == 1].index.tolist()\n\nmessage += f"From yesterday: \n"\nmessage += f"Deposited: \n"\nfor gwid in deposit_ids_today:\n    if gwid in gwid_dict:\n        message += f"{gwid} - {gwid_dict[gwid]} \n"\n\n\n#-------------------------------------------------------  STEP 4 - Not deposited with revenue negatitve today deposited part of the message  --------------------------------------------------\n# Check if the Net Revenue column contains any non-numeric values\nif not pd.to_numeric(message_p1_df_min["Net Revenue"], errors=\'coerce\').notna().all():\n    # Make the Net Revenue column into a string\n    message_p1_df_min["Net Revenue"] = message_p1_df_min["Net

In [48]:
message_p1_df

,Date,Merchant Group,Merchant,Y/N,Net Revenue,Amount Deposited,gwid
0,2025-06-24,BrightAdvantageLLC,12 - RM_12_PRIORITY_BrightAdvantageLLC_9068_75K,N,117.84,0.00,12
1,2025-06-24,BrightAdvantageLLC,13 - RM_13_PRIORITY_BrightAdvantageLLC_2323_75K,Y,103.12,109.63,13
2,2025-06-24,BrightAdvantageLLC,14 - RM_14_PRIORITY_BrightAdvantageLLC_2051_75K,Y,202.01,324.76,14
3,2025-06-24,BrightAdvantageLLC,17 - RM_17_SIGNAPAY_BrightAdvantageLLC_3881_50K,Y,332.74,1475.13,17
4,2025-06-24,BrightAdvantageLLC,18 - RM_18_SIGNAPAY_BrightAdvantageLLC_3899_50K,Y,6.06,1093.42,18
...,...,...,...,...,...,...,...
153,2025-06-24,UniquePlusLLC,42 - RM_42_NETEVIA_UniquePlusLLC_9170_65K,Y,34.69,637.14,42
154,2025-06-24,UniquePlusLLC,43 - RM_43_NETEVIA_UniquePlusLLC_9162_65K,Y,363.46,490.48,43
155,2025-06-24,UniquePlusLLC,44 - RM_44_NETEVIA_UniquePlusLLC_9154_65K,Y,156.51,724.85,44
156,2025-06-24,UniquePlusLLC,71 - HOLD_WM_71_PAYSAFE_UniquePlusLLC_9410_50K,N,-12.89,0.00,71


In [49]:
# message_p1_df is equal to message_p1_df where Date is the largest_date
message_p1_df = message_p1_df[message_p1_df["Date"] == largest_date]
message_p1_df


,Date,Merchant Group,Merchant,Y/N,Net Revenue,Amount Deposited,gwid
0,2025-06-24,BrightAdvantageLLC,12 - RM_12_PRIORITY_BrightAdvantageLLC_9068_75K,N,117.84,0.00,12
1,2025-06-24,BrightAdvantageLLC,13 - RM_13_PRIORITY_BrightAdvantageLLC_2323_75K,Y,103.12,109.63,13
2,2025-06-24,BrightAdvantageLLC,14 - RM_14_PRIORITY_BrightAdvantageLLC_2051_75K,Y,202.01,324.76,14
3,2025-06-24,BrightAdvantageLLC,17 - RM_17_SIGNAPAY_BrightAdvantageLLC_3881_50K,Y,332.74,1475.13,17
4,2025-06-24,BrightAdvantageLLC,18 - RM_18_SIGNAPAY_BrightAdvantageLLC_3899_50K,Y,6.06,1093.42,18
...,...,...,...,...,...,...,...
153,2025-06-24,UniquePlusLLC,42 - RM_42_NETEVIA_UniquePlusLLC_9170_65K,Y,34.69,637.14,42
154,2025-06-24,UniquePlusLLC,43 - RM_43_NETEVIA_UniquePlusLLC_9162_65K,Y,363.46,490.48,43
155,2025-06-24,UniquePlusLLC,44 - RM_44_NETEVIA_UniquePlusLLC_9154_65K,Y,156.51,724.85,44
156,2025-06-24,UniquePlusLLC,71 - HOLD_WM_71_PAYSAFE_UniquePlusLLC_9410_50K,N,-12.89,0.00,71


In [50]:
# From the directorty, delete the file called Master_Reconciliation.xlsx if it exists
if os.path.exists("Master_Reconciliation_GF.xlsx"):
    os.remove("Master_Reconciliation_GF.xlsx")

# Now we save the message_p1_df as Master_Reconciliation.xlsx in the same directory as the script
message_p1_df.to_excel("Master_Reconciliation_GF.xlsx", index=False)

In [51]:
deposit_reconciliation_df_copy = clean_column_names(deposit_reconciliation_df_copy)
lines = message.splitlines()
clean_message = ""
for line in lines:
    if "Report" in line and "Repeat" in line and "New" in line:
        clean_message += line
    
    if "Report" not in line and "Repeat" not in line and "New" not in line:
        gwid = line.split(" - ")
        real_gwid = int(gwid[0])
        print(real_gwid)
        for row in deposit_reconciliation_df_copy.itertuples():
            if row.gwid == real_gwid and row.net_revenue < 0:
                continue
            else:
                clean_message += line
            
        



12
15
71
72
53
74
54
76
2


In [52]:
import pandas as pd # Ensure pandas is imported

# --- Prerequisites ---
# Assume 'message' string exists.
# Assume 'deposit_reconciliation_df_copy' DataFrame exists and is cleaned.
# Ensure 'gwid' column is integer and 'net_revenue' is numeric.
try:
    deposit_reconciliation_df_copy['gwid'] = deposit_reconciliation_df_copy['gwid'].astype(int)
    deposit_reconciliation_df_copy['net_revenue'] = pd.to_numeric(deposit_reconciliation_df_copy['net_revenue'], errors='coerce')
except KeyError as e:
    print(f"Error: DataFrame is missing expected column: {e}")
    # Handle error appropriately, maybe exit or use a default
    raise # Re-raise the error if it's critical

# --- Step 1: Identify GWIDs with negative revenue (Efficient Lookup) ---
negative_revenue_gwids = set()
# Check if 'net_revenue' exists after potential coercion errors
if 'net_revenue' in deposit_reconciliation_df_copy.columns:
     negative_revenue_gwids = set(
        deposit_reconciliation_df_copy[deposit_reconciliation_df_copy['net_revenue'] <= 0]['gwid']
    )

# --- Step 2: Iterate through message lines and filter ---
original_lines = message.splitlines()
kept_lines = [] # List to store lines we want to keep

for line in original_lines:
    # Define header/special keywords
    is_header_line = "Report" in line or "Repeat" in line or "New" in line # Check if ANY keyword is present

    if is_header_line:
        kept_lines.append(line) # Keep header/special lines unconditionally
        continue # Move to the next line

    # Check if it looks like a GWID line (exactly one ' - ')
    if line.count(" - ") == 1:
        try:
            gwid_str, processor = line.split(" - ")
            gwid_str = gwid_str.strip()
            processor = processor.strip()
            real_gwid = int(gwid_str) # Try converting to integer

            # Check if this GWID should be removed (negative revenue or FlexFactor)
            if real_gwid in negative_revenue_gwids or processor == "FlexFactor":
                continue # Skip this line
            else:
                # GWID is not negative and not FlexFactor, keep this line
                kept_lines.append(line)

        except (ValueError, IndexError):
            # Handle lines where split fails or GWID part is not an integer
            print(f"Warning: Malformed GWID line, keeping it: {line}")
            kept_lines.append(line) # Keep potentially malformed lines for now
    else:
        # Line is not a header and doesn't have the 'GWID - TEXT' format
        if line.strip(): # Keep non-empty lines that don't match other patterns
             print(f"Keeping non-GWID, non-header line: {line}")
             kept_lines.append(line)

# --- Step 3: Reconstruct the message ---
clean_message = '\n'.join(kept_lines)

print(clean_message)

Report for today (2025-06-24)
Repeat: 
New: 
53 - Synovus 
74 - PAYSAFE (BBVA)(PLUSH PAY) 
54 - PAYSAFE(PNC)(PLUSH PAY) 
76 - QUANTUM (CBSL) (JORDAN) 


In [53]:
print(message)


Report for today (2025-06-24)
Repeat: 
12 - Synovus 
15 - Signapay (Esquire)(Travis) 
71 - PAYSAFE (BBVA)(PLUSH PAY) 
72 - PAYSAFE (BBVA)(PLUSH PAY) 
New: 
53 - Synovus 
74 - PAYSAFE (BBVA)(PLUSH PAY) 
54 - PAYSAFE(PNC)(PLUSH PAY) 
76 - QUANTUM (CBSL) (JORDAN) 
2 - Evolve (PayArc) 



## Exporting 

In [54]:
import pandas as pd
import numpy as np
from datetime import datetime
"""
# Generate the file name dynamically based on the current date
current_date = datetime.now().strftime("%d-%m-%Y")
file_path = f"NEF_Reconciliation_tool_output_{current_date}.xlsx"

# Create an Excel writer object
with pd.ExcelWriter(file_path, engine="xlsxwriter") as writer:
    # Write DataFrames to their respective sheets
    deposit_reconciliation_df.to_excel(writer, sheet_name="Reconciliation", index=True)
    gwid_df.to_excel(writer, sheet_name="GWIDS", index=True)
    processor_df.to_excel(writer, sheet_name="Processor", index=True)
    
    # Check if the sum of the "Amount" column in missed_df is 0
    if missed_df["Amount"].sum() != 0:
        missed_df.to_excel(writer, sheet_name="Non Matched MIDIDs", index=True)

    # Access the workbook and worksheet
    workbook = writer.book
    sheets = {
        "Reconciliation": deposit_reconciliation_df,
        "GWIDS": gwid_df,
        "Processor": processor_df
    }

    # Add "Non Matched MIDIDs" sheet if it was included
    if "Non Matched MIDIDs" in writer.sheets:
        sheets["Non Matched MIDIDs"] = missed_df

    # Define formatting styles
    bold_border_format = workbook.add_format({"bold": True, "border": 1})
    usd_format = workbook.add_format({"num_format": "$#,##0.00", "border": 1})
    bold_text_format = workbook.add_format({"bold": True, "border": 1})

    # Auto-adjust column widths and format cells
    for sheet_name, df in sheets.items():
        worksheet = writer.sheets[sheet_name]

        # Auto-fit columns
        for i, column in enumerate(df.columns):
            max_len = max(
                df[column].astype(str).map(len).max(),
                len(str(column))
            )
            worksheet.set_column(i + 1, i + 1, max_len + 2)  # i+1 because col 0 is index

        # Auto-fit index column
        max_len_index = max(df.index.astype(str).map(len).max(), len(str(df.index.name)))
        worksheet.set_column(0, 0, max_len_index + 2)

        # Format cells       
        for row_num, (idx, row) in enumerate(df.iterrows(), start=1):
            for col_num, col_name in enumerate(df.columns, start=1):
                value = row[col_name]
                if sheet_name != "Reconciliation":
                    if col_name == "CreditName" and pd.notna(value) and str(value).strip() != "":
                        worksheet.write(row_num, col_num, value, bold_border_format)
                    elif col_name == "Desc" and sheet_name == "Non Matched MIDIDs":
                        worksheet.write(row_num, col_num, value, bold_text_format)
                    elif pd.api.types.is_numeric_dtype(type(value)):
                        if pd.notna(value) and not np.isinf(value):
                            worksheet.write(row_num, col_num, value, usd_format)
                elif pd.api.types.is_numeric_dtype(type(value)):
                    if pd.notna(value) and not np.isinf(value):
                        worksheet.write(row_num, col_num, value, usd_format)

print(f"✅ Successfully exported to {file_path} with adjusted column widths, styled 'CreditName' cells, USD formatted numbers, and bold 'Desc' cells in 'Non Matched MIDIDs' (if included).")"""

'\n# Generate the file name dynamically based on the current date\ncurrent_date = datetime.now().strftime("%d-%m-%Y")\nfile_path = f"NEF_Reconciliation_tool_output_{current_date}.xlsx"\n\n# Create an Excel writer object\nwith pd.ExcelWriter(file_path, engine="xlsxwriter") as writer:\n    # Write DataFrames to their respective sheets\n    deposit_reconciliation_df.to_excel(writer, sheet_name="Reconciliation", index=True)\n    gwid_df.to_excel(writer, sheet_name="GWIDS", index=True)\n    processor_df.to_excel(writer, sheet_name="Processor", index=True)\n\n    # Check if the sum of the "Amount" column in missed_df is 0\n    if missed_df["Amount"].sum() != 0:\n        missed_df.to_excel(writer, sheet_name="Non Matched MIDIDs", index=True)\n\n    # Access the workbook and worksheet\n    workbook = writer.book\n    sheets = {\n        "Reconciliation": deposit_reconciliation_df,\n        "GWIDS": gwid_df,\n        "Processor": processor_df\n    }\n\n    # Add "Non Matched MIDIDs" sheet if it

In [55]:
missed_df

,Desc
0,NaN
Total,Total


In [56]:
import pandas as pd
import numpy as np
from datetime import datetime

# Generate the file name dynamically based on the current date
current_date = datetime.now().strftime("%d-%m-%Y")
file_path = f"GF_Reconciliation_tool_output_{current_date}.xlsx"

# Create an Excel writer object
with pd.ExcelWriter(file_path, engine="xlsxwriter") as writer:
    # Write DataFrames to their respective sheets
    deposit_reconciliation_df.to_excel(writer, sheet_name="Reconciliation", index=True)
    gwid_df.to_excel(writer, sheet_name="GWIDS", index=True)
    processor_df.to_excel(writer, sheet_name="Processor", index=True)
    
    # Check if the sum of the "Amount" column in missed_df is 0
    missed_df_total = missed_df.select_dtypes(include='number').sum().sum()
    if missed_df_total != 0:
        missed_df.to_excel(writer, sheet_name="Non Matched MIDIDs", index=True)

    # Access the workbook and worksheet
    workbook = writer.book
    sheets = {
        "Reconciliation": deposit_reconciliation_df,
        "GWIDS": gwid_df,
        "Processor": processor_df
    }

    # Add "Non Matched MIDIDs" sheet if it was included
    if "Non Matched MIDIDs" in writer.sheets:
        sheets["Non Matched MIDIDs"] = missed_df

    # Define formatting styles
    bold_border_format = workbook.add_format({"bold": True, "border": 1})
    usd_format = workbook.add_format({"num_format": "$#,##0.00", "border": 1})
    bold_text_format = workbook.add_format({"bold": True, "border": 1})

    # Auto-adjust column widths and format cells
    for sheet_name, df in sheets.items():
        worksheet = writer.sheets[sheet_name]

        # Replace NaN and Inf values in the DataFrame
        df = df.replace([np.nan, np.inf, -np.inf], "N/A")

        # Auto-fit columns
        for i, column in enumerate(df.columns):
            max_len = max(
                df[column].astype(str).map(len).max(),
                len(str(column))
            )
            worksheet.set_column(i + 1, i + 1, max_len + 2)  # i+1 because col 0 is index

        # Auto-fit index column
        max_len_index = max(df.index.astype(str).map(len).max(), len(str(df.index.name)))
        worksheet.set_column(0, 0, max_len_index + 2)

        # Format cells       
        for row_num, (idx, row) in enumerate(df.iterrows(), start=1):
            for col_num, col_name in enumerate(df.columns, start=1):
                value = row[col_name]
                if sheet_name != "Reconciliation":
                    if col_name == "CreditName" and pd.notna(value) and str(value).strip() != "":
                        worksheet.write(row_num, col_num, value, bold_border_format)
                    elif col_name == "Desc" and sheet_name == "Non Matched MIDIDs":
                        worksheet.write(row_num, col_num, value, bold_text_format)
                    elif pd.api.types.is_numeric_dtype(type(value)):
                        if pd.notna(value) and not np.isinf(value):
                            worksheet.write(row_num, col_num, value, usd_format)
                elif pd.api.types.is_numeric_dtype(type(value)):
                    if pd.notna(value) and not np.isinf(value):
                        worksheet.write(row_num, col_num, value, usd_format)

print(f"✅ Successfully exported to {file_path} with adjusted column widths, styled 'CreditName' cells, USD formatted numbers, and bold 'Desc' cells in 'Non Matched MIDIDs' (if included).")

✅ Successfully exported to GF_Reconciliation_tool_output_24-06-2025.xlsx with adjusted column widths, styled 'CreditName' cells, USD formatted numbers, and bold 'Desc' cells in 'Non Matched MIDIDs' (if included).


C:\Users\mmsou\AppData\Local\Temp\ipykernel_36028\1016421856.py:43: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace([np.nan, np.inf, -np.inf], "N/A")


In [57]:
missed_df

,Desc
0,NaN
Total,Total


In [58]:
message_p1_df.head(50)

,Date,Merchant Group,Merchant,Y/N,Net Revenue,Amount Deposited,gwid
0,2025-06-24,BrightAdvantageLLC,12 - RM_12_PRIORITY_BrightAdvantageLLC_9068_75K,N,117.84,0.00,12
1,2025-06-24,BrightAdvantageLLC,13 - RM_13_PRIORITY_BrightAdvantageLLC_2323_75K,Y,103.12,109.63,13
2,2025-06-24,BrightAdvantageLLC,14 - RM_14_PRIORITY_BrightAdvantageLLC_2051_75K,Y,202.01,324.76,14
3,2025-06-24,BrightAdvantageLLC,17 - RM_17_SIGNAPAY_BrightAdvantageLLC_3881_50K,Y,332.74,1475.13,17
4,2025-06-24,BrightAdvantageLLC,18 - RM_18_SIGNAPAY_BrightAdvantageLLC_3899_50K,Y,6.06,1093.42,18
5,2025-06-24,BrightAdvantageLLC,21 - RM_21_QUANTUM_BrightAdvantageLLC_3462_25K,Y,18.95,248.79,21
6,2025-06-24,BrightAdvantageLLC,22 - RM_22_QUANTUM_BrightAdvantageLLC_3454_25K,Y,140.92,868.49,22
7,2025-06-24,BrightAdvantageLLC,23 - RM_23_QUANTUM_BrightAdvantageLLC_3470_25K,Y,213.52,240.74,23
8,2025-06-24,BrightAdvantageLLC,24 - RM_24_QUANTUM_BrightAdvantageLLC_3488_25K,Y,533.27,822.17,24
9,2025-06-24,BrightAdvantageLLC,34 - RM_34_NETEVIA_BrightAdvantageLLC_5798_55K,Y,90.23,281.25,34


In [59]:
deposit_reconciliation_df_copy.head(50)

,date,merchant_group,merchant,yn,net_revenue,amount_deposited,gwid
0,2025-06-24,BrightAdvantageLLC,12 - RM_12_PRIORITY_BrightAdvantageLLC_9068_75K,N,-60.99,0.00,12
1,2025-06-24,BrightAdvantageLLC,13 - RM_13_PRIORITY_BrightAdvantageLLC_2323_75K,Y,202.01,177.76,13
2,2025-06-24,BrightAdvantageLLC,14 - RM_14_PRIORITY_BrightAdvantageLLC_2051_75K,Y,202.01,79.40,14
3,2025-06-24,BrightAdvantageLLC,17 - RM_17_SIGNAPAY_BrightAdvantageLLC_3881_50K,Y,648.36,473.14,17
4,2025-06-24,BrightAdvantageLLC,18 - RM_18_SIGNAPAY_BrightAdvantageLLC_3899_50K,Y,315.62,430.90,18
5,2025-06-24,BrightAdvantageLLC,21 - RM_21_QUANTUM_BrightAdvantageLLC_3462_25K,Y,117.84,210.11,21
6,2025-06-24,BrightAdvantageLLC,22 - RM_22_QUANTUM_BrightAdvantageLLC_3454_25K,Y,479.47,967.73,22
7,2025-06-24,BrightAdvantageLLC,23 - RM_23_QUANTUM_BrightAdvantageLLC_3470_25K,Y,165.68,388.13,23
8,2025-06-24,BrightAdvantageLLC,24 - RM_24_QUANTUM_BrightAdvantageLLC_3488_25K,Y,370.54,1050.39,24
9,2025-06-24,BrightAdvantageLLC,34 - RM_34_NETEVIA_BrightAdvantageLLC_5798_55K,Y,71.28,625.44,34


In [60]:
deposit_reconciliation_df_copy.tail(50)

,date,merchant_group,merchant,yn,net_revenue,amount_deposited,gwid
29,2025-06-24,DiamondSphereLLC,75 - RM_75_QUANTUM_DiamondsphereLLC_4874_25K,Y,308.66,762.30,75
30,2025-06-24,DiamondSphereLLC,76 - RM_76_QUANTUM_DiamondsphereLLC_4890_25K,Y,18.95,31.48,76
31,2025-06-24,DiamondSphereLLC,77 - RM_77_QUANTUM_DiamondsphereLLC_5293_25K,Y,510.19,1109.89,77
32,2025-06-24,DiamondSphereLLC,81 - RM_81_QUANTUM_DiamondsphereLLC_4866_25K,Y,611.93,878.91,81
33,2025-06-24,PrimeSmartSolutionsLLC,48 - RM_48_PAYARC_PrimeSmartSolutionsLLC_0345_65K,Y,534.14,636.53,48
34,2025-06-24,PrimeSmartSolutionsLLC,49 - RM_49_PAYARC_PrimeSmartSolutionsLLC_0352_65K,Y,316.64,58.79,49
35,2025-06-24,PrimeSmartSolutionsLLC,50 - RM_50_PRIORITY_PrimeSmartSolutionsLLC_057...,Y,447.63,479.30,50
36,2025-06-24,PrimeSmartSolutionsLLC,51 - RM_51_PRIORITY_PrimeSmartSolutionsLLC_520...,Y,485.79,509.70,51
37,2025-06-24,PrimeSmartSolutionsLLC,52 - RM_52_PRIORITY_PrimeSmartSolutionsLLC_956...,Y,150.45,132.84,52
38,2025-06-24,PrimeSmartSolutionsLLC,55 - RM_55_NETEVIA_PrimeSmartSolutionsLLC_3972...,Y,504.39,364.63,55


In [ ]:
deposit_reconciliation_df_copy.head(50)

,date,merchant_group,merchant,yn,net_revenue,amount_deposited,gwid
0,2025-06-24,BrightAdvantageLLC,12 - RM_12_PRIORITY_BrightAdvantageLLC_9068_75K,N,-60.99,0.00,12
1,2025-06-24,BrightAdvantageLLC,13 - RM_13_PRIORITY_BrightAdvantageLLC_2323_75K,Y,202.01,177.76,13
2,2025-06-24,BrightAdvantageLLC,14 - RM_14_PRIORITY_BrightAdvantageLLC_2051_75K,Y,202.01,79.40,14
3,2025-06-24,BrightAdvantageLLC,17 - RM_17_SIGNAPAY_BrightAdvantageLLC_3881_50K,Y,648.36,473.14,17
4,2025-06-24,BrightAdvantageLLC,18 - RM_18_SIGNAPAY_BrightAdvantageLLC_3899_50K,Y,315.62,430.90,18
5,2025-06-24,BrightAdvantageLLC,21 - RM_21_QUANTUM_BrightAdvantageLLC_3462_25K,Y,117.84,210.11,21
6,2025-06-24,BrightAdvantageLLC,22 - RM_22_QUANTUM_BrightAdvantageLLC_3454_25K,Y,479.47,967.73,22
7,2025-06-24,BrightAdvantageLLC,23 - RM_23_QUANTUM_BrightAdvantageLLC_3470_25K,Y,165.68,388.13,23
8,2025-06-24,BrightAdvantageLLC,24 - RM_24_QUANTUM_BrightAdvantageLLC_3488_25K,Y,370.54,1050.39,24
9,2025-06-24,BrightAdvantageLLC,34 - RM_34_NETEVIA_BrightAdvantageLLC_5798_55K,Y,71.28,625.44,34
